In [ ]:
📓 3_Integracao_Demografia/ │   ├── 3.1_Distribuicao_Populacional.ipynb │   ├── 3.2_Calculo_Densidade_Habitacional.ipynb │   └── 3.3_Modelagem_Padroes_Temporais.ipynb

3.1_Distribuicao_Populacional.ipynb

In [ ]:
# Distribuição Populacional: Alocação da População para Edifícios
# Este notebook distribui a população dos setores censitários para edifícios individuais

import os
import pickle
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import contextily as cx
from shapely.geometry import box
import warnings

# Ignorar avisos específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Definição dos diretórios
data_dir = '/content/drive/MyDrive/geoprocessamento_gnn/DATA'
results_dir = os.path.join(data_dir, 'results')
categorical_buildings_path = os.path.join(data_dir, 'intermediate_results', 'buildings_categorized.pkl')

# Verificando se arquivos necessários existem
if not os.path.exists(categorical_buildings_path):
    raise FileNotFoundError(f"Arquivo não encontrado: {categorical_buildings_path}. Execute os notebooks anteriores primeiro.")

# Carregando os dados de edifícios categorizados
with open(categorical_buildings_path, 'rb') as f:
    buildings = pickle.load(f)

print(f"Dados carregados: {len(buildings)} edifícios categorizados.")

In [ ]:
# Identificando e carregando os dados dos setores censitários
def load_census_data(datasets_dir, file_pattern='setores_censitarios'):
    """
    Identifica e carrega os dados dos setores censitários.
    
    Args:
        datasets_dir: Diretório que contém os datasets
        file_pattern: Padrão de nome para identificar o arquivo de setores censitários
    
    Returns:
        GeoDataFrame com os dados dos setores censitários
    """
    # Buscar arquivos que correspondem ao padrão
    census_files = []
    for root, dirs, files in os.walk(datasets_dir):
        for file in files:
            if file_pattern in file.lower() and (file.endswith('.gpkg') or file.endswith('.shp')):
                census_files.append(os.path.join(root, file))
    
    if not census_files:
        raise FileNotFoundError(f"Nenhum arquivo de setores censitários encontrado com o padrão '{file_pattern}'")
    
    # Usar o primeiro arquivo encontrado
    census_file = census_files[0]
    print(f"Carregando dados de setores censitários: {os.path.basename(census_file)}")
    
    # Identificar o tipo de arquivo e carregar adequadamente
    if census_file.endswith('.gpkg'):
        # Se for GeoPackage, listar as camadas disponíveis
        import fiona
        layers = fiona.listlayers(census_file)
        
        if len(layers) == 0:
            raise ValueError(f"Nenhuma camada encontrada no arquivo {os.path.basename(census_file)}")
        
        # Usar a primeira camada
        layer = layers[0]
        print(f"Usando camada: {layer}")
        census_data = gpd.read_file(census_file, layer=layer)
    else:
        # Se for Shapefile
        census_data = gpd.read_file(census_file)
    
    # Verificar se o CRS é compatível com os edifícios
    if census_data.crs != buildings.crs:
        print(f"Transformando CRS de setores censitários para compatibilidade: {census_data.crs} -> {buildings.crs}")
        census_data = census_data.to_crs(buildings.crs)
    
    # Explorar os dados carregados
    print(f"Dados carregados: {len(census_data)} setores censitários")
    print(f"Colunas disponíveis: {census_data.columns.tolist()}")
    
    return census_data

# Carregar os dados dos setores censitários
census_data = load_census_data(data_dir)

In [ ]:
# Identificando as colunas populacionais nos dados censitários
def identify_population_columns(census_gdf):
    """
    Identifica colunas relacionadas à população nos dados censitários.
    
    Args:
        census_gdf: GeoDataFrame com os dados dos setores censitários
    
    Returns:
        Dicionário com colunas populacionais identificadas
    """
    population_columns = {}
    
    # Palavras-chave para buscar em nomes de colunas
    keywords = {
        'total_population': ['populacao', 'pop_total', 'est_populacao', 'pop_', 'habitantes', 'pessoas'],
        'male_population': ['pop_masc', 'homens', 'masculino'],
        'female_population': ['pop_fem', 'mulheres', 'feminino'],
        'household_count': ['domicilios', 'residencias', 'habitacoes'],
        'density': ['densidade', 'dens_pop', 'densidade_pop'],
        'age_groups': ['faixa_', 'idade_', 'pop_0', 'pop_1', 'pop_2', 'pop_3', 'pop_4', 'pop_5', 'pop_6', 'pop_7', 'pop_8', 'pop_9']
    }
    
    # Buscar por correspondências nos nomes das colunas
    for category, search_terms in keywords.items():
        for col in census_gdf.columns:
            col_lower = col.lower()
            if any(term in col_lower for term in search_terms):
                if category == 'age_groups':
                    if 'age_groups' not in population_columns:
                        population_columns['age_groups'] = []
                    population_columns['age_groups'].append(col)
                else:
                    population_columns[category] = col
                    break
    
    # Verificar se encontramos a coluna principal de população
    if 'total_population' not in population_columns:
        print("AVISO: Não foi possível identificar a coluna de população total.")
        print("Colunas numéricas disponíveis:")
        numeric_cols = census_gdf.select_dtypes(include=['int64', 'float64']).columns.tolist()
        for col in numeric_cols:
            print(f"- {col}: {census_gdf[col].max()}")
        
        # Tentar usar a primeira coluna numérica com valores razoáveis
        for col in numeric_cols:
            if census_gdf[col].max() > 100 and census_gdf[col].min() >= 0:
                population_columns['total_population'] = col
                print(f"Usando coluna '{col}' como população total.")
                break
    
    # Exibir as colunas identificadas
    print("\nColunas populacionais identificadas:")
    for category, col in population_columns.items():
        if category != 'age_groups':
            print(f"- {category}: {col}")
        else:
            print(f"- {category}: {len(col)} colunas encontradas")
    
    return population_columns

# Identificar colunas populacionais
population_columns = identify_population_columns(census_data)

# Verificar e imprimir estatísticas básicas da população
if 'total_population' in population_columns:
    pop_col = population_columns['total_population']
    print(f"\nEstatísticas da população total ({pop_col}):")
    print(f"- Total: {census_data[pop_col].sum():,.0f} habitantes")
    print(f"- Média por setor: {census_data[pop_col].mean():,.2f} habitantes")
    print(f"- Mínimo: {census_data[pop_col].min():,.0f}")
    print(f"- Máximo: {census_data[pop_col].max():,.0f}")
else:
    raise ValueError("Não foi possível identificar a coluna de população total. Verifique os dados censitários.")

In [ ]:
# Verificar e visualizar os dados antes da distribuição
def explore_input_data(buildings_gdf, census_gdf, pop_column):
    """
    Explora e visualiza os dados de entrada antes da distribuição populacional.
    
    Args:
        buildings_gdf: GeoDataFrame com os edifícios
        census_gdf: GeoDataFrame com os setores censitários
        pop_column: Nome da coluna de população total
    """
    # Verificar a sobreposição espacial entre os conjuntos de dados
    buildings_bbox = box(*buildings_gdf.total_bounds)
    census_bbox = box(*census_gdf.total_bounds)
    
    overlap = buildings_bbox.intersection(census_bbox).area / buildings_bbox.union(census_bbox).area
    print(f"Sobreposição espacial entre os conjuntos de dados: {overlap:.2%}")
    
    # Visualizar os dois conjuntos de dados
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Plotar setores censitários com gradiente de população
    census_gdf.plot(column=pop_column, ax=ax, alpha=0.5, cmap='viridis', legend=True,
                   legend_kwds={'label': f'População ({pop_column})', 'orientation': 'horizontal'})
    
    # Amostrar edifícios para não sobrecarregar a visualização
    sample_size = min(5000, len(buildings_gdf))
    buildings_sample = buildings_gdf.sample(sample_size)
    
    # Plotar edifícios por categoria
    if 'building_class_enhanced' in buildings_gdf.columns:
        buildings_sample.plot(column='building_class_enhanced', ax=ax, 
                              markersize=5, categorical=True, legend=False)
    else:
        buildings_sample.plot(ax=ax, color='red', markersize=5, alpha=0.5)
    
    # Adicionar mapa base
    try:
        cx.add_basemap(ax, crs=buildings_gdf.crs)
    except Exception as e:
        print(f"Não foi possível adicionar o mapa base: {e}")
    
    # Configurar o gráfico
    ax.set_title('Setores Censitários e Edifícios', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Analisar os tipos de edifícios para a distribuição populacional
    print("\nAnálise dos tipos de edifícios para distribuição populacional:")
    
    # Identificar edifícios residenciais
    building_col = next((col for col in ['building', 'building_type', 'building_class_enhanced'] 
                         if col in buildings_gdf.columns), None)
    
    if building_col:
        # Contar tipos de edifícios
        building_counts = buildings_gdf[building_col].value_counts()
        
        # Identificar tipos residenciais
        residential_keywords = ['residencial', 'residential', 'house', 'apartamento', 'apartments', 'habitacional', 'dwelling']
        residential_types = [t for t in building_counts.index if any(kw in str(t).lower() for kw in residential_keywords)]
        
        print(f"Tipos de edifícios identificados como residenciais:")
        for res_type in residential_types:
            print(f"- {res_type}: {building_counts[res_type]} edifícios")
        
        # Quantificar edifícios potencialmente residenciais
        residential_count = sum(buildings_gdf[building_col].isin(residential_types))
        print(f"\nTotal de edifícios potencialmente residenciais: {residential_count} ({residential_count/len(buildings_gdf):.2%})")
    else:
        print("Não foi possível identificar uma coluna de tipos de edifícios.")

# Explorar os dados de entrada
pop_column = population_columns['total_population']
explore_input_data(buildings, census_data, pop_column)

In [ ]:
# Determinar quais edifícios são residenciais
def identify_residential_buildings(buildings_gdf):
    """
    Identifica edifícios residenciais para distribuição populacional.
    
    Args:
        buildings_gdf: GeoDataFrame com os edifícios
    
    Returns:
        GeoDataFrame com coluna adicional indicando se o edifício é residencial
    """
    buildings_copy = buildings_gdf.copy()
    
    # Adicionar coluna para indicar se o edifício é residencial
    buildings_copy['is_residential'] = False
    
    # Palavras-chave que indicam uso residencial
    residential_keywords = [
        'residencial', 'residential', 'house', 'apartamento', 'apartments', 
        'habitacional', 'dwelling', 'casa', 'moradia', 'housing',
        'residencia', 'residence', 'home', 'domestic', 'domiciliar'
    ]
    
    # Colunas a verificar para uso residencial
    cols_to_check = [
        'building_class_enhanced', 'categoria_funcional', 'building', 
        'tipo', 'type', 'amenity', 'land_category', 'landuse'
    ]
    
    # Verificar em cada coluna se há indicação de uso residencial
    for col in cols_to_check:
        if col in buildings_copy.columns:
            # Marcar como residencial se alguma palavra-chave for encontrada
            mask = buildings_copy[col].notna() & buildings_copy[col].astype(str).str.lower().apply(
                lambda x: any(kw in x for kw in residential_keywords)
            )
            buildings_copy.loc[mask, 'is_residential'] = True
    
    # Caso específico: se tiver coluna 'building_class_enhanced' com classificação específica
    if 'building_class_enhanced' in buildings_copy.columns:
        residential_classes = [
            'residencial_unifamiliar', 'residencial_multifamiliar', 
            'residencial_misto', 'residencial'
        ]
        mask = buildings_copy['building_class_enhanced'].isin(residential_classes)
        buildings_copy.loc[mask, 'is_residential'] = True
    
    # Estatísticas sobre edifícios residenciais identificados
    residential_count = buildings_copy['is_residential'].sum()
    print(f"Edifícios residenciais identificados: {residential_count} ({residential_count/len(buildings_copy):.2%})")
    
    # Se nenhum edifício residencial for identificado, emitir aviso
    if residential_count == 0:
        print("AVISO: Nenhum edifício residencial identificado!")
        # Como alternativa, considerar todos os edifícios
        buildings_copy['is_residential'] = True
        print("Considerando todos os edifícios como potencialmente residenciais para distribuição populacional.")
    
    return buildings_copy

# Identificar edifícios residenciais
buildings_with_residential = identify_residential_buildings(buildings)

In [ ]:
# Identificar critérios para pesos de distribuição populacional
def identify_distribution_weights(buildings_gdf):
    """
    Identifica critérios para pesos na distribuição populacional.
    
    Args:
        buildings_gdf: GeoDataFrame com os edifícios
    
    Returns:
        Dicionário com colunas a usar como pesos e estratégia de distribuição
    """
    weight_columns = {}
    
    # Verificar coluna de área
    area_cols = [col for col in buildings_gdf.columns if 'area' in col.lower()]
    if area_cols:
        print("Colunas de área encontradas:")
        for col in area_cols:
            non_zero = (buildings_gdf[col] > 0).sum()
            print(f"- {col}: {non_zero} valores não-zero ({non_zero/len(buildings_gdf):.2%})")
        
        # Selecionar a coluna de área com mais valores não-zero
        best_area_col = max(area_cols, key=lambda c: (buildings_gdf[c] > 0).sum())
        weight_columns['area'] = best_area_col
    
    # Verificar coluna de volume
    volume_cols = [col for col in buildings_gdf.columns if 'volume' in col.lower()]
    if volume_cols:
        print("\nColunas de volume encontradas:")
        for col in volume_cols:
            non_zero = (buildings_gdf[col] > 0).sum()
            print(f"- {col}: {non_zero} valores não-zero ({non_zero/len(buildings_gdf):.2%})")
        
        # Selecionar a coluna de volume com mais valores não-zero
        best_volume_col = max(volume_cols, key=lambda c: (buildings_gdf[c] > 0).sum())
        weight_columns['volume'] = best_volume_col
    
    # Verificar coluna de altura ou número de andares
    height_cols = [col for col in buildings_gdf.columns 
                  if any(term in col.lower() for term in ['height', 'altura', 'levels', 'andar', 'pavimento'])]
    if height_cols:
        print("\nColunas de altura/andares encontradas:")
        for col in height_cols:
            non_zero = (buildings_gdf[col] > 0).sum()
            print(f"- {col}: {non_zero} valores não-zero ({non_zero/len(buildings_gdf):.2%})")
        
        # Selecionar a coluna de altura com mais valores não-zero
        best_height_col = max(height_cols, key=lambda c: (buildings_gdf[c] > 0).sum())
        weight_columns['height'] = best_height_col
    
    # Determinar estratégia de distribuição
    strategy = 'area'  # Padrão: usar área
    
    if 'volume' in weight_columns and (buildings_gdf[weight_columns['volume']] > 0).mean() > 0.5:
        # Se temos volume para mais de 50% dos edifícios, usar volume
        strategy = 'volume'
        print("\nEstratégia selecionada: Distribuição por VOLUME (melhor representação do espaço habitável)")
    
    elif 'area' in weight_columns and 'height' in weight_columns:
        # Se temos área e altura, usar produto (aproximação de volume)
        strategy = 'area_height'
        print("\nEstratégia selecionada: Distribuição por ÁREA x ALTURA (aproximação de volume)")
    
    elif 'area' in weight_columns:
        # Se só temos área, usar área
        strategy = 'area'
        print("\nEstratégia selecionada: Distribuição por ÁREA (única métrica disponível)")
    
    else:
        # Se não temos nenhuma métrica, usar contagem simples
        strategy = 'count'
        print("\nEstratégia selecionada: Distribuição por CONTAGEM (sem métricas dimensionais disponíveis)")
    
    weight_columns['strategy'] = strategy
    
    return weight_columns

# Identificar critérios para pesos
weight_criteria = identify_distribution_weights(buildings_with_residential)

In [ ]:
# Realizar junção espacial entre edifícios e setores censitários
def spatial_join_buildings_census(buildings_gdf, census_gdf):
    """
    Realiza a junção espacial entre edifícios e setores censitários.
    
    Args:
        buildings_gdf: GeoDataFrame com os edifícios
        census_gdf: GeoDataFrame com os setores censitários
    
    Returns:
        GeoDataFrame dos edifícios com informações do setor censitário
    """
    print("Realizando junção espacial entre edifícios e setores censitários...")
    
    # Verificar se os CRS são compatíveis
    if buildings_gdf.crs != census_gdf.crs:
        print(f"Transformando CRS dos setores censitários para compatibilidade")
        census_gdf = census_gdf.to_crs(buildings_gdf.crs)
    
    # Selecionar colunas relevantes dos setores censitários para a junção
    census_cols = ['geometry']
    if 'total_population' in population_columns:
        census_cols.append(population_columns['total_population'])
    
    # Adicionar outras colunas demograficamente relevantes
    for cat, col in population_columns.items():
        if cat != 'age_groups' and cat != 'total_population':
            census_cols.append(col)
    
    # Adicionar colunas de faixa etária, se disponíveis
    if 'age_groups' in population_columns:
        census_cols.extend(population_columns['age_groups'])
    
    # Garantir que não haja colunas duplicadas
    census_cols = list(dict.fromkeys(census_cols))
    
    # Realizar a junção espacial
    buildings_census = gpd.sjoin(buildings_gdf, census_gdf[census_cols], how='left', predicate='within')
    
    # Verificar resultados da junção
    matched = buildings_census[population_columns['total_population']].notna().sum()
    not_matched = buildings_census[population_columns['total_population']].isna().sum()
    
    print(f"Edifícios associados a setores censitários: {matched} ({matched/len(buildings_gdf):.2%})")
    
    if not_matched > 0:
        print(f"Edifícios não associados a setores censitários: {not_matched} ({not_matched/len(buildings_gdf):.2%})")
        
        # Para edifícios não associados por 'within', tentar com 'intersects'
        if not_matched / len(buildings_gdf) > 0.05:  # Se mais de 5% não foi associado
            print("Tentando associar edifícios restantes usando interseção...")
            
            # Identificar edifícios não associados
            unmatched_buildings = buildings_gdf[~buildings_gdf.index.isin(buildings_census.dropna(subset=[population_columns['total_population']]).index)]
            
            # Realizar junção por interseção para os não associados
            if len(unmatched_buildings) > 0:
                buildings_intersect = gpd.sjoin(unmatched_buildings, census_gdf[census_cols], how='left', predicate='intersects')
                
                # Adicionar os edifícios associados por interseção
                matched_by_intersect = buildings_intersect[population_columns['total_population']].notna().sum()
                
                if matched_by_intersect > 0:
                    # Mesclar os resultados
                    buildings_census = pd.concat([
                        buildings_census.dropna(subset=[population_columns['total_population']]),
                        buildings_intersect.dropna(subset=[population_columns['total_population']])
                    ])
                    
                    print(f"Edifícios associados por interseção: {matched_by_intersect}")
                    print(f"Total de edifícios associados: {len(buildings_census)} ({len(buildings_census)/len(buildings_gdf):.2%})")
    
    return buildings_census

# Realizar junção espacial
buildings_with_census = spatial_join_buildings_census(buildings_with_residential, census_data)

In [ ]:
# Calcular pesos para distribuição populacional
def calculate_distribution_weights(buildings_census, weight_criteria):
    """
    Calcula pesos para distribuição populacional com base na estratégia selecionada.
    
    Args:
        buildings_census: GeoDataFrame dos edifícios com dados censitários
        weight_criteria: Dicionário com critérios de peso
    
    Returns:
        GeoDataFrame com pesos calculados
    """
    buildings_weights = buildings_census.copy()
    
    # Adicionar coluna de peso
    buildings_weights['weight'] = 0.0
    
    # Filtrar apenas edifícios residenciais associados a setores censitários
    pop_col = population_columns['total_population']
    mask_residential = buildings_weights['is_residential'] & buildings_weights[pop_col].notna()
    
    # Se não houver edifícios residenciais associados, usar todos os edifícios
    if mask_residential.sum() == 0:
        print("AVISO: Nenhum edifício identificado como residencial com dados censitários.")
        mask_residential = buildings_weights[pop_col].notna()
        print(f"Usando todos os {mask_residential.sum()} edifícios com dados censitários.")
    else:
        print(f"Usando {mask_residential.sum()} edifícios residenciais com dados censitários.")
    
    # Calcular pesos com base na estratégia selecionada
    strategy = weight_criteria['strategy']
    
    if strategy == 'volume' and 'volume' in weight_criteria:
        print(f"Calculando pesos usando volume ({weight_criteria['volume']})")
        volume_col = weight_criteria['volume']
        
        # Garantir que não haja valores negativos ou nulos
        buildings_weights.loc[mask_residential & (buildings_weights[volume_col] <= 0), volume_col] = buildings_weights.loc[mask_residential, volume_col].median()
        
        # Usar volume como peso
        buildings_weights.loc[mask_residential, 'weight'] = buildings_weights.loc[mask_residential, volume_col]
    
    elif strategy == 'area_height' and 'area' in weight_criteria and 'height' in weight_criteria:
        print(f"Calculando pesos usando área ({weight_criteria['area']}) × altura ({weight_criteria['height']})")
        area_col = weight_criteria['area']
        height_col = weight_criteria['height']
        
        # Garantir que não haja valores negativos ou nulos
        buildings_weights.loc[mask_residential & (buildings_weights[area_col] <= 0), area_col] = buildings_weights.loc[mask_residential, area_col].median()
        buildings_weights.loc[mask_residential & (buildings_weights[height_col] <= 0), height_col] = 1.0
        
        # Usar área × altura como peso (aproximação de volume)
        buildings_weights.loc[mask_residential, 'weight'] = buildings_weights.loc[mask_residential, area_col] * buildings_weights.loc[mask_residential, height_col]
    
    elif strategy == 'area' and 'area' in weight_criteria:
        print(f"Calculando pesos usando área ({weight_criteria['area']})")
        area_col = weight_criteria['area']
        
        # Garantir que não haja valores negativos ou nulos
        buildings_weights.loc[mask_residential & (buildings_weights[area_col] <= 0), area_col] = buildings_weights.loc[mask_residential, area_col].median()
        
        # Usar área como peso
        buildings_weights.loc[mask_residential, 'weight'] = buildings_weights.loc[mask_residential, area_col]
    
    else:
        print("Calculando pesos usando contagem simples (todos os edifícios recebem peso igual)")
        # Usar contagem simples (peso igual para todos os edifícios)
        buildings_weights.loc[mask_residential, 'weight'] = 1.0
    
    # Verificar se há pesos válidos
    valid_weights = (buildings_weights['weight'] > 0).sum()
    
    if valid_weights == 0:
        print("AVISO: Nenhum peso válido calculado. Usando contagem simples.")
        buildings_weights.loc[mask_residential, 'weight'] = 1.0
    else:
        print(f"Pesos válidos calculados para {valid_weights} edifícios.")
    
    # Estatísticas dos pesos
    if valid_weights > 0:
        print("\nEstatísticas dos pesos:")
        weight_stats = buildings_weights.loc[buildings_weights['weight'] > 0, 'weight'].describe()
        for stat, value in weight_stats.items():
            print(f"- {stat}: {value:.2f}")
    
    return buildings_weights

# Calcular pesos para distribuição
buildings_with_weights = calculate_distribution_weights(buildings_with_census, weight_criteria)

In [ ]:
# Distribuir a população para os edifícios
def distribute_population(buildings_weights, pop_column):
    """
    Distribui a população dos setores censitários para os edifícios com base nos pesos.
    
    Args:
        buildings_weights: GeoDataFrame com pesos calculados
        pop_column: Coluna de população total
    
    Returns:
        GeoDataFrame com população distribuída
    """
    print("Distribuindo população para os edifícios...")
    
    # Copiar o GeoDataFrame
    buildings_population = buildings_weights.copy()
    
    # Criar coluna para a população distribuída
    buildings_population['estimated_population'] = 0.0
    
    # Obter o índice do setor censitário para agrupar
    census_index = buildings_population.columns.get_loc('index_right') if 'index_right' in buildings_population.columns else None
    
    if census_index is None:
        print("AVISO: Coluna 'index_right' não encontrada após a junção espacial.")
        # Tentar encontrar o índice de outra forma
        join_suffix = "_right"
        index_cols = [col for col in buildings_population.columns if col.endswith(join_suffix)]
        
        if index_cols:
            # Usar a primeira coluna como índice
            census_id_col = index_cols[0]
            print(f"Usando coluna '{census_id_col}' como identificador do setor censitário.")
        else:
            # Criar ID temporário baseado no setor censitário
            print("Criando ID temporário para os setores censitários...")
            buildings_population['temp_census_id'] = buildings_population.groupby(pop_column).ngroup()
            census_id_col = 'temp_census_id'
    else:
        census_id_col = 'index_right'
    
    # Distribuir a população por setor censitário
    for census_id, group in tqdm(buildings_population.groupby(census_id_col), 
                              desc="Processando setores censitários"):
        # Verificar se há edifícios com peso neste grupo
        valid_group = group[group['weight'] > 0]
        
        if len(valid_group) == 0:
            continue
        
        # Obter a população total do setor
        if pop_column in valid_group.columns:
            total_pop = valid_group[pop_column].iloc[0]
            
            # Verificar se a população é válida
            if pd.isna(total_pop) or total_pop <= 0:
                continue
            
            # Calcular a soma dos pesos no setor
            sum_weights = valid_group['weight'].sum()
            
            if sum_weights > 0:
                # Distribuir proporcionalmente ao peso
                for idx in valid_group.index:
                    buildings_population.at[idx, 'estimated_population'] = (
                        valid_group.at[idx, 'weight'] / sum_weights * total_pop
                    )
    
    # Arredondar para número inteiro de pessoas (preservando o total)
    # Esta técnica mantém o total arredondando os maiores resíduos para cima
    buildings_population['temp_floor'] = np.floor(buildings_population['estimated_population']).astype(int)
    buildings_population['temp_residual'] = buildings_population['estimated_population'] - buildings_population['temp_floor']
    
    # Distribuir a população por setor censitário para garantir que o total seja mantido
    for census_id, group in buildings_population.groupby(census_id_col):
        # Calcular quantas pessoas estão faltando para atingir o total do setor
        if pop_column in group.columns:
            total_pop = group[pop_column].iloc[0] if len(group) > 0 else 0
            
            if pd.isna(total_pop) or total_pop <= 0:
                continue
            
            # Pessoas já alocadas (floor)
            allocated = group['temp_floor'].sum()
            
            # Pessoas faltando para atingir o total
            missing = int(round(total_pop - allocated))
            
            if missing > 0:
                # Ordenar por residual descendente
                residual_order = group.sort_values('temp_residual', ascending=False).index[:missing]
                
                # Adicionar 1 para os maiores residuais
                buildings_population.loc[residual_order, 'temp_floor'] += 1
    
    # Atualizar a população estimada com os valores arredondados
    buildings_population['estimated_population'] = buildings_population['temp_floor']
    
    # Remover colunas temporárias
    buildings_population = buildings_population.drop(columns=['temp_floor', 'temp_residual'])
    
    # Verificar resultados da distribuição
    total_distributed = buildings_population['estimated_population'].sum()
    total_census = buildings_population[pop_column].dropna().iloc[0] if len(buildings_population) > 0 else 0
    
    print(f"\nResultados da distribuição populacional:")
    print(f"- População total nos setores censitários: {total_census:,.0f}")
    print(f"- População total distribuída: {total_distributed:,.0f}")
    print(f"- Diferença: {total_distributed - total_census:,.0f} ({(total_distributed - total_census)/total_census*100:.2f}%)")
    
    # Estatísticas da população distribuída para edifícios residenciais
    res_pop = buildings_population[buildings_population['is_residential']]['estimated_population']
    if len(res_pop) > 0:
        print("\nEstatísticas de população por edifício residencial:")
        print(f"- Média: {res_pop.mean():.2f} pessoas/edifício")
        print(f"- Mediana: {res_pop.median():.2f} pessoas/edifício")
        print(f"- Máximo: {res_pop.max():.2f} pessoas/edifício")
        print(f"- Mínimo: {res_pop[res_pop > 0].min():.2f} pessoas/edifício")
    
    return buildings_population

# Distribuir a população
buildings_with_population = distribute_population(buildings_with_weights, population_columns['total_population'])

In [ ]:
# Visualizar resultados da distribuição populacional
def visualize_population_distribution(buildings_population, census_data, pop_column):
    """
    Visualiza os resultados da distribuição populacional.
    
    Args:
        buildings_population: GeoDataFrame com população distribuída
        census_data: GeoDataFrame com setores censitários
        pop_column: Coluna de população total
    """
    # Criar figura com dois mapas lado a lado
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    
    # Mapa 1: Setores censitários com população original
    census_data.plot(column=pop_column, ax=axs[0], alpha=0.5, cmap='viridis', legend=True,
                   legend_kwds={'label': f'População Original ({pop_column})', 'orientation': 'horizontal'})
    
    # Adicionar título
    axs[0].set_title('População por Setor Censitário (Original)', fontsize=14)
    
    # Mapa 2: Edifícios com população distribuída
    # Utilizar tamanho proporcional à população
    buildings_with_pop = buildings_population[buildings_population['estimated_population'] > 0]
    
    if len(buildings_with_pop) > 0:
        # Normalizar o tamanho dos pontos para melhor visualização
        min_pop = buildings_with_pop['estimated_population'].min()
        max_pop = buildings_with_pop['estimated_population'].max()
        
        # Função para calcular tamanho do ponto (entre 5 e 100)
        def point_size(pop):
            if max_pop == min_pop:
                return 10
            return 5 + (pop - min_pop) / (max_pop - min_pop) * 95
        
        # Aplicar função para calcular tamanho
        buildings_with_pop['point_size'] = buildings_with_pop['estimated_population'].apply(point_size)
        
        # Plotar edifícios com tamanho proporcional à população
        buildings_with_pop.plot(ax=axs[1], markersize='point_size', column='estimated_population',
                              alpha=0.7, cmap='plasma', legend=True,
                              legend_kwds={'label': 'População Distribuída', 'orientation': 'horizontal'})
        
        # Adicionar título
        axs[1].set_title('População Distribuída por Edifício', fontsize=14)
    else:
        axs[1].set_title('Nenhum edifício com população distribuída', fontsize=14)
    
    # Adicionar mapa base em ambos
    try:
        for ax in axs:
            cx.add_basemap(ax, crs=buildings_population.crs)
    except Exception as e:
        print(f"Não foi possível adicionar o mapa base: {e}")
    
    # Ajustar layout
    plt.tight_layout()
    plt.show()
    
    # Gráficos adicionais para análise da distribuição
    plt.figure(figsize=(10, 6))
    
    # Histograma da população por edifício
    sns.histplot(buildings_population[buildings_population['estimated_population'] > 0]['estimated_population'], 
                 bins=30, kde=True)
    
    plt.title('Distribuição da População por Edifício')
    plt.xlabel('Habitantes por Edifício')
    plt.ylabel('Frequência')
    plt.tight_layout()
    plt.show()

# Visualizar resultados
visualize_population_distribution(buildings_with_population, census_data, population_columns['total_population'])

In [ ]:
# Armazenar os resultados da distribuição populacional
def save_population_distribution(buildings_population, filename='buildings_population.gpkg'):
    """
    Salva os resultados da distribuição populacional para uso em análises subsequentes.
    
    Args:
        buildings_population: GeoDataFrame com população distribuída
        filename: Nome do arquivo para salvamento
    
    Returns:
        Caminho para o arquivo salvo
    """
    # Criar pasta de resultados intermediários se não existir
    results_dir = os.path.join(data_dir, 'intermediate_results')
    os.makedirs(results_dir, exist_ok=True)
    
    # Caminho completo para o arquivo
    result_path = os.path.join(results_dir, filename)
    
    # Salvar como GeoPackage
    buildings_population.to_file(result_path, driver='GPKG')
    
    # Salvar também como pickle para preservar tipos de dados
    pickle_path = os.path.join(results_dir, 'buildings_population.pkl')
    with open(pickle_path, 'wb') as f:
        pickle.dump(buildings_population, f)
    
    print(f"Resultados da distribuição populacional salvos em:\n- {result_path}\n- {pickle_path}")
    
    # Criar um relatório resumido em CSV
    report_data = {
        'building_type': [],
        'count': [],
        'total_population': [],
        'mean_population': [],
        'population_percentage': []
    }
    
    # Identificar coluna de tipo de edifício
    building_col = next((col for col in ['building_class_enhanced', 'building', 'categoria_funcional'] 
                      if col in buildings_population.columns), None)
    
    if building_col:
        # Agrupar por tipo de edifício
        for building_type, group in buildings_population.groupby(building_col):
            pop_sum = group['estimated_population'].sum()
            
            report_data['building_type'].append(building_type)
            report_data['count'].append(len(group))
            report_data['total_population'].append(pop_sum)
            report_data['mean_population'].append(pop_sum / len(group) if len(group) > 0 else 0)
            report_data['population_percentage'].append(pop_sum / buildings_population['estimated_population'].sum() * 100 if buildings_population['estimated_population'].sum() > 0 else 0)
    
        # Criar DataFrame e salvar como CSV
        report_df = pd.DataFrame(report_data)
        report_df = report_df.sort_values('total_population', ascending=False)
        
        report_path = os.path.join(results_dir, 'population_distribution_report.csv')
        report_df.to_csv(report_path, index=False)
        
        print(f"Relatório resumido salvo em: {report_path}")
    
    return result_path, pickle_path

# Salvar os resultados
gpkg_path, pickle_path = save_population_distribution(buildings_with_population)

In [ ]:
# Resumo e conclusão
print("="*80)
print("RESUMO DA DISTRIBUIÇÃO POPULACIONAL")
print("="*80)

# Estatísticas gerais
total_buildings = len(buildings_with_population)
residential_buildings = buildings_with_population['is_residential'].sum()
buildings_with_pop = (buildings_with_population['estimated_population'] > 0).sum()
total_pop = buildings_with_population['estimated_population'].sum()

print(f"Total de edifícios processados: {total_buildings}")
print(f"Edifícios identificados como residenciais: {residential_buildings} ({residential_buildings/total_buildings*100:.2f}%)")
print(f"Edifícios com população atribuída: {buildings_with_pop} ({buildings_with_pop/total_buildings*100:.2f}%)")
print(f"População total distribuída: {total_pop:,.0f}")

# Identificar coluna de tipo de edifício
building_col = next((col for col in ['building_class_enhanced', 'building', 'categoria_funcional'] 
                   if col in buildings_with_population.columns), None)

if building_col:
    # Top 5 tipos de edifícios por população
    pop_by_type = buildings_with_population.groupby(building_col)['estimated_population'].sum().sort_values(ascending=False)
    
    print("\nTop 5 tipos de edifícios por população:")
    for i, (building_type, pop) in enumerate(pop_by_type.head(5).items()):
        print(f"{i+1}. {building_type}: {pop:,.0f} habitantes ({pop/total_pop*100:.2f}%)")

# Benefícios da distribuição populacional
print("\nBenefícios da distribuição populacional:")
print("1. Estimativa detalhada da população em escala de edifício")
print("2. Base para análises avançadas de exposição a riscos e planejamento urbano")
print("3. Identificação de hotspots populacionais de alta resolução")
print("4. Suporte para modelagem de padrões temporais de ocupação")

# Próximos passos
print("\nPróximos passos:")
print("1. Cálculo de métricas de densidade habitacional")
print("2. Modelagem de padrões temporais de ocupação por tipo de edifício")
print("3. Análises de acessibilidade a serviços essenciais")

print(f"\nOs resultados da distribuição populacional foram salvos e estão prontos para o próximo notebook:")
print(f"- GeoPackage: {os.path.basename(gpkg_path)}")
print(f"- Pickle: {os.path.basename(pickle_path)}")
print("="*80)

3.2_Calculo_Densidade_Habitacional.ipynb

In [ ]:
# Cálculo de Densidade Habitacional
# Este notebook calcula métricas de densidade habitacional a partir da população distribuída

import os
import pickle
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import contextily as cx
from matplotlib.colors import LinearSegmentedColormap
import warnings
from matplotlib.colors import TwoSlopeNorm
from shapely.geometry import box

# Ignorar avisos específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Definição dos diretórios
data_dir = '/content/drive/MyDrive/geoprocessamento_gnn/DATA'
results_dir = os.path.join(data_dir, 'intermediate_results')
buildings_population_path = os.path.join(results_dir, 'buildings_population.pkl')

# Verificando se arquivos necessários existem
if not os.path.exists(buildings_population_path):
    raise FileNotFoundError(f"Arquivo não encontrado: {buildings_population_path}. Execute o notebook 3.1_Distribuicao_Populacional.ipynb primeiro.")

# Carregando os dados de edifícios com população
with open(buildings_population_path, 'rb') as f:
    buildings_population = pickle.load(f)

print(f"Dados carregados: {len(buildings_population)} edifícios com dados populacionais.")

In [ ]:
# Verificar os dados carregados
def explore_population_data(buildings_gdf):
    """
    Explora os dados de população distribuída para análise de densidade.
    
    Args:
        buildings_gdf: GeoDataFrame com edifícios e população distribuída
    """
    # Verificar se a coluna de população estimada existe
    if 'estimated_population' not in buildings_gdf.columns:
        raise ValueError("Coluna 'estimated_population' não encontrada. Verifique se a distribuição populacional foi realizada corretamente.")
    
    # Estatísticas básicas da população
    pop_stats = buildings_gdf['estimated_population'].describe()
    
    print("Estatísticas da população distribuída:")
    for stat, value in pop_stats.items():
        print(f"- {stat}: {value:.2f}")
    
    # Edifícios com população atribuída
    buildings_with_pop = buildings_gdf[buildings_gdf['estimated_population'] > 0]
    
    print(f"\nEdifícios com população atribuída: {len(buildings_with_pop)} ({len(buildings_with_pop)/len(buildings_gdf)*100:.2f}%)")
    print(f"População total: {buildings_gdf['estimated_population'].sum():,.0f} habitantes")
    
    # Verificar colunas disponíveis para cálculo de densidade
    area_cols = [col for col in buildings_gdf.columns if 'area' in col.lower()]
    volume_cols = [col for col in buildings_gdf.columns if 'volume' in col.lower()]
    
    print("\nColunas disponíveis para cálculo de densidade:")
    
    print("Colunas de área:")
    for col in area_cols:
        coverage = buildings_gdf[col].notna().mean() * 100
        print(f"- {col}: {coverage:.2f}% de cobertura")
    
    print("\nColunas de volume:")
    for col in volume_cols:
        coverage = buildings_gdf[col].notna().mean() * 100
        print(f"- {col}: {coverage:.2f}% de cobertura")
    
    # Histograma da população por edifício
    plt.figure(figsize=(10, 6))
    sns.histplot(buildings_with_pop['estimated_population'], bins=30, kde=True)
    plt.title('Distribuição da População por Edifício')
    plt.xlabel('Habitantes por Edifício')
    plt.ylabel('Frequência')
    plt.tight_layout()
    plt.show()

# Explorar os dados carregados
explore_population_data(buildings_population)

In [ ]:
# Identificar colunas relevantes para cálculo de densidade
def identify_density_columns(buildings_gdf):
    """
    Identifica as colunas relevantes para cálculo de densidade habitacional.
    
    Args:
        buildings_gdf: GeoDataFrame com edifícios e população distribuída
    
    Returns:
        Dicionário com colunas identificadas para cada tipo de densidade
    """
    density_columns = {}
    
    # Identificar melhor coluna de área
    area_candidates = [
        'area_m2', 'area', 'footprint_area', 'building_area',
        'roof_area_m2', 'roof_area', 'floor_area'
    ]
    
    # Verificar quais colunas existem e têm dados
    existing_area_cols = []
    for col in area_candidates:
        if col in buildings_gdf.columns and buildings_gdf[col].notna().any():
            coverage = buildings_gdf[col].notna().mean() * 100
            non_zero = (buildings_gdf[col] > 0).mean() * 100
            existing_area_cols.append((col, coverage, non_zero))
    
    # Selecionar a melhor coluna de área (com maior cobertura e valores não-zero)
    if existing_area_cols:
        # Ordenar por cobertura e depois por valores não-zero
        existing_area_cols.sort(key=lambda x: (x[1], x[2]), reverse=True)
        best_area_col = existing_area_cols[0][0]
        density_columns['area'] = best_area_col
        print(f"Melhor coluna de área identificada: {best_area_col} ({existing_area_cols[0][1]:.2f}% de cobertura)")
    else:
        print("Nenhuma coluna de área identificada. Tentando buscar colunas alternativas...")
        # Buscar colunas que possam conter área
        for col in buildings_gdf.columns:
            if 'area' in col.lower() and buildings_gdf[col].notna().any():
                density_columns['area'] = col
                coverage = buildings_gdf[col].notna().mean() * 100
                print(f"Coluna de área alternativa: {col} ({coverage:.2f}% de cobertura)")
                break
    
    # Identificar melhor coluna de volume ou aproximação de volume
    volume_candidates = [
        'volume_m3', 'volume', 'building_volume'
    ]
    
    # Verificar quais colunas existem e têm dados
    existing_volume_cols = []
    for col in volume_candidates:
        if col in buildings_gdf.columns and buildings_gdf[col].notna().any():
            coverage = buildings_gdf[col].notna().mean() * 100
            non_zero = (buildings_gdf[col] > 0).mean() * 100
            existing_volume_cols.append((col, coverage, non_zero))
    
    # Selecionar a melhor coluna de volume
    if existing_volume_cols:
        existing_volume_cols.sort(key=lambda x: (x[1], x[2]), reverse=True)
        best_volume_col = existing_volume_cols[0][0]
        density_columns['volume'] = best_volume_col
        print(f"Melhor coluna de volume identificada: {best_volume_col} ({existing_volume_cols[0][1]:.2f}% de cobertura)")
    else:
        print("Nenhuma coluna de volume identificada.")
        
        # Verificar se temos altura/andares para aproximação de volume
        height_cols = [col for col in buildings_gdf.columns 
                      if any(term in col.lower() for term in ['height', 'altura', 'levels', 'andar', 'pavimento'])]
        
        if height_cols and 'area' in density_columns:
            # Usar a primeira coluna de altura encontrada
            height_col = height_cols[0]
            density_columns['height'] = height_col
            
            # Calcular aproximação de volume (área × altura)
            print(f"Calculando aproximação de volume a partir de {density_columns['area']} × {height_col}")
            
            # Verificar cobertura da coluna de altura
            coverage = buildings_gdf[height_col].notna().mean() * 100
            print(f"Coluna de altura: {height_col} ({coverage:.2f}% de cobertura)")
    
    # Identificar coluna de tipo de edifício para análises específicas
    type_candidates = ['building_class_enhanced', 'categoria_funcional', 'building', 'type']
    
    for col in type_candidates:
        if col in buildings_gdf.columns and buildings_gdf[col].notna().any():
            density_columns['type'] = col
            coverage = buildings_gdf[col].notna().mean() * 100
            print(f"Coluna de tipo de edifício: {col} ({coverage:.2f}% de cobertura)")
            break
    
    return density_columns

# Identificar colunas para cálculo de densidade
density_columns = identify_density_columns(buildings_population)

# Verificar se temos as colunas necessárias
if 'area' not in density_columns:
    raise ValueError("Não foi possível identificar uma coluna de área. Não é possível calcular densidades habitacionais.")

In [ ]:
# Calcular métricas de densidade habitacional
def calculate_density_metrics(buildings_gdf, density_columns):
    """
    Calcula métricas de densidade habitacional a partir da população distribuída.
    
    Args:
        buildings_gdf: GeoDataFrame com edifícios e população distribuída
        density_columns: Dicionário com colunas para cálculo de densidade
    
    Returns:
        GeoDataFrame com métricas de densidade calculadas
    """
    # Copiar o GeoDataFrame para não modificar o original
    buildings_density = buildings_gdf.copy()
    
    # Filtrar apenas edifícios com população
    buildings_with_pop = buildings_density[buildings_density['estimated_population'] > 0]
    
    print(f"Calculando métricas de densidade para {len(buildings_with_pop)} edifícios com população...")
    
    # 1. Densidade por área (habitantes/m²) - métrica básica
    area_col = density_columns['area']
    
    # Verificar se há valores nulos ou negativos
    invalid_area = (buildings_with_pop[area_col].isna()) | (buildings_with_pop[area_col] <= 0)
    if invalid_area.any():
        print(f"AVISO: {invalid_area.sum()} edifícios têm área inválida (nula ou não positiva).")
        
        # Usar área média para substituir valores inválidos
        median_area = buildings_with_pop.loc[~invalid_area, area_col].median()
        buildings_density.loc[invalid_area & (buildings_density['estimated_population'] > 0), area_col] = median_area
        print(f"Substituindo áreas inválidas pela mediana: {median_area:.2f} m²")
    
    # Calcular densidade por área
    buildings_density['density_per_area'] = buildings_density['estimated_population'] / buildings_density[area_col]
    
    # 2. Densidade por volume (habitantes/m³), se disponível
    if 'volume' in density_columns:
        volume_col = density_columns['volume']
        
        # Verificar valores inválidos
        invalid_volume = (buildings_with_pop[volume_col].isna()) | (buildings_with_pop[volume_col] <= 0)
        if invalid_volume.any():
            print(f"AVISO: {invalid_volume.sum()} edifícios têm volume inválido (nulo ou não positivo).")
            
            # Usar volume médio para substituir valores inválidos
            median_volume = buildings_with_pop.loc[~invalid_volume, volume_col].median()
            buildings_density.loc[invalid_volume & (buildings_density['estimated_population'] > 0), volume_col] = median_volume
            print(f"Substituindo volumes inválidos pela mediana: {median_volume:.2f} m³")
        
        # Calcular densidade por volume
        buildings_density['density_per_volume'] = buildings_density['estimated_population'] / buildings_density[volume_col]
    
    # Alternativa: densidade aproximada por área × altura, se disponível
    elif 'height' in density_columns:
        height_col = density_columns['height']
        
        # Verificar valores inválidos
        invalid_height = (buildings_with_pop[height_col].isna()) | (buildings_with_pop[height_col] <= 0)
        if invalid_height.any():
            print(f"AVISO: {invalid_height.sum()} edifícios têm altura inválida (nula ou não positiva).")
            
            # Usar altura média para substituir valores inválidos
            median_height = buildings_with_pop.loc[~invalid_height, height_col].median()
            buildings_density.loc[invalid_height & (buildings_density['estimated_population'] > 0), height_col] = median_height
            print(f"Substituindo alturas inválidas pela mediana: {median_height:.2f}")
        
        # Calcular volume aproximado (área × altura)
        buildings_density['approx_volume'] = buildings_density[area_col] * buildings_density[height_col]
        
        # Calcular densidade por volume aproximado
        buildings_density['density_per_volume'] = buildings_density['estimated_population'] / buildings_density['approx_volume']
    
    # 3. Densidade por unidade habitacional (ocupação média), se possível inferir unidades
    # Supõe-se que cada unidade habitacional tenha em média X m² (valor típico para residências)
    avg_unit_area = 60  # Valor médio típico para unidades habitacionais (ajustar conforme necessário)
    
    # Estimativa do número de unidades habitacionais
    if 'height' in density_columns:
        # Se temos altura, considerar o número de pavimentos
        buildings_density['estimated_floors'] = buildings_density[height_col].map(lambda h: max(1, int(h / 3)))  # Estimativa de 3m por pavimento
        buildings_density['estimated_units'] = (buildings_density[area_col] * buildings_density['estimated_floors'] / avg_unit_area).round()
    else:
        # Senão, assumir que toda a área é ocupada por unidades (edifício térreo)
        buildings_density['estimated_units'] = (buildings_density[area_col] / avg_unit_area).round()
    
    # Evitar unidades zeradas
    buildings_density.loc[buildings_density['estimated_units'] <= 0, 'estimated_units'] = 1
    
    # Calcular densidade por unidade (ocupação média)
    buildings_density['occupancy_per_unit'] = buildings_density['estimated_population'] / buildings_density['estimated_units']
    
    # 4. Classificar edifícios por nível de densidade
    # Criar intervalos de densidade por área para classificação
    density_breaks = [0, 0.005, 0.01, 0.02, 0.05, float('inf')]  # habitantes/m²
    density_labels = ['Muito baixa', 'Baixa', 'Média', 'Alta', 'Muito alta']
    
    # Classificar usando pd.cut
    buildings_density['density_class'] = pd.cut(
        buildings_density['density_per_area'], 
        bins=density_breaks, 
        labels=density_labels,
        include_lowest=True
    )
    
    # 5. Calcular índice de densidade normalizado (0-1)
    # Usar método de normalização min-max
    valid_density = buildings_density[buildings_density['density_per_area'] > 0]['density_per_area']
    
    if len(valid_density) > 0:
        min_density = valid_density.quantile(0.01)  # Usar 1º percentil para evitar outliers
        max_density = valid_density.quantile(0.99)  # Usar 99º percentil para evitar outliers
        
        # Normalizar entre 0 e 1
        buildings_density['density_index'] = buildings_density['density_per_area'].map(
            lambda x: max(0, min(1, (x - min_density) / (max_density - min_density))) if x > 0 else 0
        )
    
    # Exibir estatísticas das métricas calculadas
    print("\nEstatísticas das métricas de densidade:")
    
    metrics = ['density_per_area', 'density_per_volume', 'occupancy_per_unit', 'density_index']
    available_metrics = [m for m in metrics if m in buildings_density.columns]
    
    for metric in available_metrics:
        valid_values = buildings_density[buildings_density[metric] > 0][metric]
        
        if len(valid_values) > 0:
            print(f"\n{metric}:")
            stats = valid_values.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
            for stat, value in stats.items():
                print(f"- {stat}: {value:.6f}")
    
    # Distribuição de classes de densidade
    if 'density_class' in buildings_density.columns:
        class_counts = buildings_density['density_class'].value_counts()
        print("\nDistribuição de classes de densidade:")
        for cls, count in class_counts.items():
            print(f"- {cls}: {count} edifícios ({count/len(buildings_density)*100:.2f}%)")
    
    return buildings_density

# Calcular métricas de densidade
buildings_with_density = calculate_density_metrics(buildings_population, density_columns)

In [ ]:
# Analisar densidade por tipo de edifício
def analyze_density_by_building_type(buildings_density, density_columns):
    """
    Analisa as métricas de densidade por tipo de edifício.
    
    Args:
        buildings_density: GeoDataFrame com métricas de densidade calculadas
        density_columns: Dicionário com colunas para análise
    """
    if 'type' not in density_columns:
        print("Coluna de tipo de edifício não identificada. Não é possível analisar por tipo.")
        return
    
    type_col = density_columns['type']
    
    # Considerar apenas edifícios com população
    buildings_with_pop = buildings_density[buildings_density['estimated_population'] > 0]
    
    # Agrupar por tipo de edifício
    type_groups = buildings_with_pop.groupby(type_col)
    
    # Criar DataFrame para armazenar estatísticas por tipo
    type_stats = []
    
    for building_type, group in type_groups:
        if pd.isna(building_type) or len(group) == 0:
            continue
        
        # Estatísticas básicas
        stats = {
            'building_type': building_type,
            'count': len(group),
            'total_population': group['estimated_population'].sum(),
            'mean_population': group['estimated_population'].mean()
        }
        
        # Adicionar estatísticas de densidade
        if 'density_per_area' in group.columns:
            stats['mean_density_area'] = group['density_per_area'].mean()
            stats['median_density_area'] = group['density_per_area'].median()
        
        if 'density_per_volume' in group.columns:
            stats['mean_density_volume'] = group['density_per_volume'].mean()
            stats['median_density_volume'] = group['density_per_volume'].median()
        
        if 'occupancy_per_unit' in group.columns:
            stats['mean_occupancy'] = group['occupancy_per_unit'].mean()
            stats['median_occupancy'] = group['occupancy_per_unit'].median()
        
        # Distribuição de classes de densidade
        if 'density_class' in group.columns:
            class_distribution = group['density_class'].value_counts(normalize=True).to_dict()
            for cls, proportion in class_distribution.items():
                stats[f'class_{cls}'] = proportion
        
        type_stats.append(stats)
    
    # Converter para DataFrame
    type_stats_df = pd.DataFrame(type_stats)
    
    # Ordenar por população total
    if 'total_population' in type_stats_df.columns:
        type_stats_df = type_stats_df.sort_values('total_population', ascending=False)
    
    # Exibir estatísticas
    print("Estatísticas de densidade por tipo de edifício:")
    display(type_stats_df.head(10))
    
    # Visualizar comparações
    if len(type_stats_df) > 1:
        # 1. Gráfico de barras para população média por tipo
        plt.figure(figsize=(12, 6))
        
        # Limitar a 15 tipos para melhor visualização
        plot_df = type_stats_df.head(15).copy()
        
        # Criar gráfico
        ax = sns.barplot(x='building_type', y='mean_population', data=plot_df)
        plt.title('População Média por Tipo de Edifício', fontsize=14)
        plt.xlabel('Tipo de Edifício')
        plt.ylabel('População Média')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
        
        # 2. Densidade média por tipo (se disponível)
        if 'mean_density_area' in plot_df.columns:
            plt.figure(figsize=(12, 6))
            ax = sns.barplot(x='building_type', y='mean_density_area', data=plot_df)
            plt.title('Densidade Média (hab/m²) por Tipo de Edifício', fontsize=14)
            plt.xlabel('Tipo de Edifício')
            plt.ylabel('Densidade Média (hab/m²)')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
        
        # 3. Ocupação média por tipo (se disponível)
        if 'mean_occupancy' in plot_df.columns:
            plt.figure(figsize=(12, 6))
            ax = sns.barplot(x='building_type', y='mean_occupancy', data=plot_df)
            plt.title('Ocupação Média por Unidade Habitacional por Tipo de Edifício', fontsize=14)
            plt.xlabel('Tipo de Edifício')
            plt.ylabel('Ocupação Média (hab/unidade)')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
    
    return type_stats_df

# Analisar densidade por tipo de edifício
type_density_stats = analyze_density_by_building_type(buildings_with_density, density_columns)

In [ ]:
# Visualizar a distribuição espacial de densidade
def visualize_density_distribution(buildings_density, variable='density_per_area', title=None):
    """
    Visualiza a distribuição espacial de densidade habitacional.
    
    Args:
        buildings_density: GeoDataFrame com métricas de densidade calculadas
        variable: Coluna de densidade a visualizar
        title: Título do mapa (opcional)
    """
    # Filtrar edifícios com densidade válida
    valid_buildings = buildings_density[(buildings_density[variable] > 0) & buildings_density[variable].notna()]
    
    if len(valid_buildings) == 0:
        print(f"Nenhum edifício com valores válidos para '{variable}'")
        return
    
    # Limitar valores extremos para melhor visualização
    min_val = valid_buildings[variable].quantile(0.01)
    max_val = valid_buildings[variable].quantile(0.99)
    
    # Criar figura
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Definir paleta de cores personalizada
    cmap = LinearSegmentedColormap.from_list(
        'density', 
        ['#f2f0f7', '#cbc9e2', '#9e9ac8', '#6a51a3', '#320064'], 
        N=256
    )
    
    # Calcular tamanho dos pontos baseado na população (para representação visual)
    max_pop = valid_buildings['estimated_population'].max()
    min_pop = valid_buildings['estimated_population'].min()
    
    if max_pop > min_pop:
        valid_buildings['point_size'] = valid_buildings['estimated_population'].map(
            lambda p: 5 + (p - min_pop) / (max_pop - min_pop) * 95
        )
    else:
        valid_buildings['point_size'] = 10
    
    # Plotar mapa de densidade
    im = valid_buildings.plot(
        column=variable,
        cmap=cmap,
        markersize='point_size',
        ax=ax,
        alpha=0.8,
        vmin=min_val,
        vmax=max_val,
        legend=True,
        legend_kwds={
            'label': f"Densidade Habitacional ({variable})",
            'orientation': 'horizontal',
            'shrink': 0.8,
            'pad': 0.01
        }
    )
    
    # Adicionar mapa base
    try:
        cx.add_basemap(ax, crs=buildings_density.crs)
    except Exception as e:
        print(f"Não foi possível adicionar o mapa base: {e}")
    
    # Adicionar título
    if title:
        ax.set_title(title, fontsize=16)
    else:
        ax.set_title(f'Distribuição Espacial de Densidade Habitacional ({variable})', fontsize=16)
    
    plt.tight_layout()
    plt.show()
    
    # Mapa adicional para classes de densidade (se disponível)
    if 'density_class' in buildings_density.columns:
        # Criar figura
        fig, ax = plt.subplots(figsize=(15, 15))
        
        # Plotar mapa de classes de densidade
        buildings_density.plot(
            column='density_class',
            categorical=True,
            cmap='viridis',
            markersize='point_size' if 'point_size' in valid_buildings.columns else 10,
            ax=ax,
            alpha=0.8,
            legend=True,
            legend_kwds={
                'title': "Classe de Densidade",
                'loc': 'upper left',
                'bbox_to_anchor': (1, 1)
            }
        )
        
        # Adicionar mapa base
        try:
            cx.add_basemap(ax, crs=buildings_density.crs)
        except Exception as e:
            print(f"Não foi possível adicionar o mapa base: {e}")
        
        # Adicionar título
        ax.set_title('Classificação de Densidade Habitacional', fontsize=16)
        
        plt.tight_layout()
        plt.show()

# Visualizar mapa de densidade habitacional
if 'density_per_area' in buildings_with_density.columns:
    visualize_density_distribution(buildings_with_density, 'density_per_area', 'Densidade Habitacional (hab/m²)')

# Visualizar mapa de ocupação por unidade habitacional
if 'occupancy_per_unit' in buildings_with_density.columns:
    visualize_density_distribution(buildings_with_density, 'occupancy_per_unit', 'Ocupação por Unidade Habitacional')

In [ ]:
# Calcular métricas de densidade por célula/grade para análise espacial
def calculate_grid_density(buildings_gdf, cell_size=100):
    """
    Cria uma grade regular e calcula métricas de densidade habitacional por célula.
    
    Args:
        buildings_gdf: GeoDataFrame com edifícios e população
        cell_size: Tamanho da célula da grade em metros
    
    Returns:
        GeoDataFrame com células da grade e métricas de densidade
    """
    # Verificar se o CRS é projetado (necessário para cálculos em metros)
    if not buildings_gdf.crs or not buildings_gdf.crs.is_projected:
        print("CRS não é projetado. Transformando para sistema de coordenadas projetado (UTM)...")
        buildings_gdf = buildings_gdf.to_crs(epsg=31983)  # UTM 23S (SIRGAS 2000)
    
    # Criar grade regular
    xmin, ymin, xmax, ymax = buildings_gdf.total_bounds
    
    # Ajustar os limites para múltiplos de cell_size
    xmin = np.floor(xmin / cell_size) * cell_size
    ymin = np.floor(ymin / cell_size) * cell_size
    xmax = np.ceil(xmax / cell_size) * cell_size
    ymax = np.ceil(ymax / cell_size) * cell_size
    
    # Criar sequências de coordenadas
    x_coords = np.arange(xmin, xmax + cell_size, cell_size)
    y_coords = np.arange(ymin, ymax + cell_size, cell_size)
    
    # Criar células da grade
    cells = []
    cell_id = 0
    
    for x in x_coords[:-1]:
        for y in y_coords[:-1]:
            # Criar polígono da célula
            cell = box(x, y, x + cell_size, y + cell_size)
            
            cells.append({
                'cell_id': cell_id,
                'geometry': cell
            })
            
            cell_id += 1
    
    # Criar GeoDataFrame da grade
    grid = gpd.GeoDataFrame(cells, crs=buildings_gdf.crs)
    
    print(f"Grade criada com {len(grid)} células de {cell_size}m × {cell_size}m")
    
    # Realizar junção espacial com edifícios
    buildings_in_cells = gpd.sjoin(
        buildings_gdf[buildings_gdf['estimated_population'] > 0], 
        grid, 
        how='inner', 
        predicate='intersects'
    )
    
    # Agrupar por célula e calcular métricas
    cell_metrics = []
    
    for cell_id, group in tqdm(buildings_in_cells.groupby('cell_id'), desc="Calculando métricas por célula"):
        # Extrair geometria da célula
        cell_geom = grid.loc[grid['cell_id'] == cell_id, 'geometry'].iloc[0]
        
        # Calcular métricas
        metrics = {
            'cell_id': cell_id,
            'building_count': len(group),
            'population': group['estimated_population'].sum(),
            'population_density': group['estimated_population'].sum() / (cell_size * cell_size / 1e6),  # hab/km²
            'geometry': cell_geom
        }
        
        # Adicionar estatísticas de edifícios
        if 'density_per_area' in group.columns:
            metrics['mean_building_density'] = group['density_per_area'].mean()
        
        if 'occupancy_per_unit' in group.columns:
            metrics['mean_occupancy'] = group['occupancy_per_unit'].mean()
        
        cell_metrics.append(metrics)
    
    # Criar GeoDataFrame com métricas por célula
    grid_metrics = gpd.GeoDataFrame(cell_metrics, crs=buildings_gdf.crs)
    
    # Adicionar classificação de densidade populacional
    if 'population_density' in grid_metrics.columns:
        # Definir intervalos para classificação
        density_breaks = [0, 1000, 5000, 10000, 20000, float('inf')]  # hab/km²
        density_labels = ['Muito baixa', 'Baixa', 'Média', 'Alta', 'Muito alta']
        
        # Classificar usando pd.cut
        grid_metrics['density_class'] = pd.cut(
            grid_metrics['population_density'], 
            bins=density_breaks, 
            labels=density_labels,
            include_lowest=True
        )
    
    # Exibir estatísticas da grade
    if 'population' in grid_metrics.columns:
        print("\nEstatísticas de densidade habitacional por célula:")
        print(f"- Células com população: {(grid_metrics['population'] > 0).sum()} ({(grid_metrics['population'] > 0).sum()/len(grid_metrics)*100:.2f}%)")
        print(f"- População total: {grid_metrics['population'].sum():,.0f}")
        print(f"- Densidade média: {grid_metrics['population_density'].mean():,.2f} hab/km²")
        print(f"- Densidade máxima: {grid_metrics['population_density'].max():,.2f} hab/km²")
    
    # Distribuição de classes de densidade
    if 'density_class' in grid_metrics.columns:
        class_counts = grid_metrics['density_class'].value_counts()
        print("\nDistribuição de classes de densidade por célula:")
        for cls, count in class_counts.items():
            print(f"- {cls}: {count} células ({count/(grid_metrics['population'] > 0).sum()*100:.2f}% das células habitadas)")
    
    return grid_metrics

# Calcular densidade por grade regular
grid_density = calculate_grid_density(buildings_with_density, cell_size=200)

In [ ]:
# Visualizar métricas de densidade por célula
def visualize_grid_density(grid_metrics):
    """
    Visualiza as métricas de densidade habitacional por célula da grade.
    
    Args:
        grid_metrics: GeoDataFrame com células da grade e métricas de densidade
    """
    # Verificar se temos dados de população
    if 'population' not in grid_metrics.columns:
        print("Dados de população por célula não encontrados.")
        return
    
    # Filtrar células com população
    inhabited_cells = grid_metrics[grid_metrics['population'] > 0]
    
    if len(inhabited_cells) == 0:
        print("Nenhuma célula com população encontrada.")
        return
    
    # 1. Mapa de densidade populacional
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Definir diverging colormap para densidade
    pop_min = inhabited_cells['population_density'].min()
    pop_max = inhabited_cells['population_density'].max()
    pop_median = inhabited_cells['population_density'].median()
    
    # Criar norma centralizada na mediana
    norm = TwoSlopeNorm(vmin=pop_min, vcenter=pop_median, vmax=pop_max)
    
    # Plotar mapa de densidade
    inhabited_cells.plot(
        column='population_density',
        cmap='viridis',
        ax=ax,
        alpha=0.7,
        norm=norm,
        legend=True,
        legend_kwds={
            'label': "Densidade Populacional (hab/km²)",
            'orientation': 'horizontal',
            'shrink': 0.8,
            'pad': 0.01
        }
    )
    
    # Adicionar mapa base
    try:
        cx.add_basemap(ax, crs=grid_metrics.crs)
    except Exception as e:
        print(f"Não foi possível adicionar o mapa base: {e}")
    
    # Adicionar título
    ax.set_title('Densidade Populacional por Célula (hab/km²)', fontsize=16)
    
    plt.tight_layout()
    plt.show()
    
    # 2. Mapa de classes de densidade
    if 'density_class' in grid_metrics.columns:
        fig, ax = plt.subplots(figsize=(15, 15))
        
        # Plotar mapa de classes de densidade
        inhabited_cells.plot(
            column='density_class',
            categorical=True,
            cmap='plasma',
            ax=ax,
            alpha=0.7,
            legend=True,
            legend_kwds={
                'title': "Classe de Densidade",
                'loc': 'upper left',
                'bbox_to_anchor': (1, 1)
            }
        )
        
        # Adicionar mapa base
        try:
            cx.add_basemap(ax, crs=grid_metrics.crs)
        except Exception as e:
            print(f"Não foi possível adicionar o mapa base: {e}")
        
        # Adicionar título
        ax.set_title('Classificação de Densidade Populacional por Célula', fontsize=16)
        
        plt.tight_layout()
        plt.show()
    
    # 3. Mapa de contagem de edifícios
    if 'building_count' in grid_metrics.columns:
        fig, ax = plt.subplots(figsize=(15, 15))
        
        # Plotar mapa de contagem de edifícios
        inhabited_cells.plot(
            column='building_count',
            cmap='YlOrRd',
            ax=ax,
            alpha=0.7,
            legend=True,
            legend_kwds={
                'label': "Número de Edifícios",
                'orientation': 'horizontal',
                'shrink': 0.8,
                'pad': 0.01
            }
        )
        
        # Adicionar mapa base
        try:
            cx.add_basemap(ax, crs=grid_metrics.crs)
        except Exception as e:
            print(f"Não foi possível adicionar o mapa base: {e}")
        
        # Adicionar título
        ax.set_title('Número de Edifícios por Célula', fontsize=16)
        
        plt.tight_layout()
        plt.show()

# Visualizar densidade por grade
visualize_grid_density(grid_density)

In [ ]:
# Calcular estatísticas de ocupação por dormitório
def calculate_occupancy_statistics(buildings_gdf):
    """
    Calcula estatísticas de ocupação por dormitório com base nas métricas de densidade.
    
    Args:
        buildings_gdf: GeoDataFrame com métricas de densidade calculadas
    
    Returns:
        DataFrame com estatísticas de ocupação por dormitório
    """
    # Verificar se temos dados de ocupação por unidade
    if 'occupancy_per_unit' not in buildings_gdf.columns:
        print("Dados de ocupação por unidade não encontrados.")
        return None
    
    # Filtrar apenas edifícios residenciais com população
    residential_buildings = buildings_gdf[(buildings_gdf['is_residential'] == True) & 
                                         (buildings_gdf['estimated_population'] > 0)]
    
    if len(residential_buildings) == 0:
        print("Nenhum edifício residencial com população encontrado.")
        return None
    
    # Definir parâmetros para estimativa de dormitórios
    # Supõe-se que cada unidade tenha um certo número de dormitórios conforme a área
    
    # Função para estimar número de dormitórios com base na área da unidade
    def estimate_bedrooms(unit_area):
        if unit_area < 40:
            return 1  # Estúdio ou 1 dormitório
        elif unit_area < 70:
            return 2  # 2 dormitórios
        elif unit_area < 100:
            return 3  # 3 dormitórios
        else:
            return 4  # 4 ou mais dormitórios
    
    # Estimar área média por unidade
    area_col = density_columns['area']
    
    if 'estimated_units' in residential_buildings.columns and 'estimated_floors' in residential_buildings.columns:
        # Calcular área média por unidade
        residential_buildings['unit_area'] = residential_buildings[area_col] * residential_buildings['estimated_floors'] / residential_buildings['estimated_units']
    else:
        # Usar área total dividida pelo número estimado de unidades
        residential_buildings['unit_area'] = residential_buildings[area_col] / residential_buildings['estimated_units']
    
    # Estimar número de dormitórios
    residential_buildings['estimated_bedrooms'] = residential_buildings['unit_area'].apply(estimate_bedrooms)
    
    # Calcular ocupação por dormitório
    residential_buildings['occupancy_per_bedroom'] = residential_buildings['estimated_population'] / (residential_buildings['estimated_bedrooms'] * residential_buildings['estimated_units'])
    
    # Corrigir valores inválidos
    residential_buildings.loc[residential_buildings['occupancy_per_bedroom'].isna() | 
                          (residential_buildings['occupancy_per_bedroom'] <= 0) |
                          (residential_buildings['occupancy_per_bedroom'] > 10), 'occupancy_per_bedroom'] = np.nan
    
    # Estatísticas de ocupação por dormitório
    bedroom_stats = residential_buildings['occupancy_per_bedroom'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
    
    print("Estatísticas de ocupação por dormitório:")
    for stat, value in bedroom_stats.items():
        print(f"- {stat}: {value:.4f}")
    
    # Classificar ocupação por dormitório
    # Definir intervalos para classificação
    occupancy_breaks = [0, 0.5, 1.0, 1.5, 2.0, float('inf')]
    occupancy_labels = ['Muito baixa', 'Baixa', 'Normal', 'Alta', 'Muito alta']
    
    # Classificar usando pd.cut
    residential_buildings['bedroom_occupancy_class'] = pd.cut(
        residential_buildings['occupancy_per_bedroom'], 
        bins=occupancy_breaks, 
        labels=occupancy_labels,
        include_lowest=True
    )
    
    # Resumo da classificação
    occupancy_class_stats = residential_buildings['bedroom_occupancy_class'].value_counts()
    
    print("\nDistribuição de classes de ocupação por dormitório:")
    for cls, count in occupancy_class_stats.items():
        print(f"- {cls}: {count} edifícios ({count/len(residential_buildings)*100:.2f}%)")
    
    # Analisar por tipo de edifício, se disponível
    if 'type' in density_columns:
        type_col = density_columns['type']
        
        # Agrupar por tipo de edifício
        type_occupancy = residential_buildings.groupby(type_col)['occupancy_per_bedroom'].agg(['mean', 'median', 'std', 'count']).reset_index()
        type_occupancy = type_occupancy.sort_values('mean', ascending=False)
        
        print("\nOcupação média por dormitório por tipo de edifício:")
        display(type_occupancy.head(10))
    
    return residential_buildings

# Calcular estatísticas de ocupação por dormitório
buildings_with_occupancy = calculate_occupancy_statistics(buildings_with_density)

In [ ]:
# Salvar os resultados do cálculo de densidade habitacional
def save_density_metrics(buildings_gdf, grid_metrics=None, filename='buildings_density.gpkg'):
    """
    Salva os resultados do cálculo de densidade habitacional para uso em análises subsequentes.
    
    Args:
        buildings_gdf: GeoDataFrame com métricas de densidade calculadas
        grid_metrics: GeoDataFrame com métricas de densidade por célula (opcional)
        filename: Nome do arquivo para salvamento
    
    Returns:
        Caminhos para os arquivos salvos
    """
    # Criar pasta de resultados intermediários se não existir
    results_dir = os.path.join(data_dir, 'intermediate_results')
    os.makedirs(results_dir, exist_ok=True)
    
    # Caminho completo para o arquivo de edifícios
    buildings_path = os.path.join(results_dir, filename)
    
    # Salvar edifícios como GeoPackage
    buildings_gdf.to_file(buildings_path, driver='GPKG')
    
    # Salvar também como pickle para preservar tipos de dados
    buildings_pickle_path = os.path.join(results_dir, 'buildings_density.pkl')
    with open(buildings_pickle_path, 'wb') as f:
        pickle.dump(buildings_gdf, f)
    
    print(f"Resultados de densidade habitacional por edifício salvos em:\n- {buildings_path}\n- {buildings_pickle_path}")
    
    # Salvar grade se disponível
    grid_paths = None
    if grid_metrics is not None:
        # Caminho para o arquivo da grade
        grid_path = os.path.join(results_dir, 'density_grid.gpkg')
        
        # Salvar grade como GeoPackage
        grid_metrics.to_file(grid_path, driver='GPKG')
        
        # Salvar também como pickle
        grid_pickle_path = os.path.join(results_dir, 'density_grid.pkl')
        with open(grid_pickle_path, 'wb') as f:
            pickle.dump(grid_metrics, f)
        
        print(f"Resultados de densidade habitacional por célula salvos em:\n- {grid_path}\n- {grid_pickle_path}")
        
        grid_paths = (grid_path, grid_pickle_path)
    
    # Criar um relatório resumido em CSV
    report_data = {}
    
    # Estatísticas de densidade
    if 'density_per_area' in buildings_gdf.columns:
        density_stats = buildings_gdf[buildings_gdf['density_per_area'] > 0]['density_per_area'].describe()
        for stat, value in density_stats.items():
            report_data[f'density_area_{stat}'] = value
    
    if 'density_per_volume' in buildings_gdf.columns:
        volume_stats = buildings_gdf[buildings_gdf['density_per_volume'] > 0]['density_per_volume'].describe()
        for stat, value in volume_stats.items():
            report_data[f'density_volume_{stat}'] = value
    
    if 'occupancy_per_unit' in buildings_gdf.columns:
        occupancy_stats = buildings_gdf[buildings_gdf['occupancy_per_unit'] > 0]['occupancy_per_unit'].describe()
        for stat, value in occupancy_stats.items():
            report_data[f'occupancy_unit_{stat}'] = value
    
    # Estatísticas por classe de densidade
    if 'density_class' in buildings_gdf.columns:
        class_counts = buildings_gdf['density_class'].value_counts()
        for cls, count in class_counts.items():
            report_data[f'class_{cls}'] = count
    
    # Criar DataFrame e salvar como CSV
    report_df = pd.DataFrame([report_data])
    report_path = os.path.join(results_dir, 'density_metrics_report.csv')
    report_df.to_csv(report_path, index=False)
    
    print(f"Relatório resumido salvo em: {report_path}")
    
    return (buildings_path, buildings_pickle_path), grid_paths

# Salvar os resultados
buildings_paths, grid_paths = save_density_metrics(buildings_with_density, grid_density)

In [ ]:
# Resumo e conclusão
print("="*80)
print("RESUMO DAS MÉTRICAS DE DENSIDADE HABITACIONAL")
print("="*80)

# Estatísticas gerais
total_buildings = len(buildings_with_density)
buildings_with_pop = (buildings_with_density['estimated_population'] > 0).sum()
total_pop = buildings_with_density['estimated_population'].sum()

print(f"Total de edifícios processados: {total_buildings}")
print(f"Edifícios com população atribuída: {buildings_with_pop} ({buildings_with_pop/total_buildings*100:.2f}%)")
print(f"População total: {total_pop:,.0f}")

# Estatísticas de densidade
if 'density_per_area' in buildings_with_density.columns:
    valid_density = buildings_with_density[buildings_with_density['density_per_area'] > 0]['density_per_area']
    mean_density = valid_density.mean()
    median_density = valid_density.median()
    max_density = valid_density.max()
    
    print(f"\nDensidade habitacional (hab/m²):")
    print(f"- Média: {mean_density:.6f}")
    print(f"- Mediana: {median_density:.6f}")
    print(f"- Máxima: {max_density:.6f}")

# Estatísticas de ocupação
if 'occupancy_per_unit' in buildings_with_density.columns:
    valid_occupancy = buildings_with_density[buildings_with_density['occupancy_per_unit'] > 0]['occupancy_per_unit']
    mean_occupancy = valid_occupancy.mean()
    median_occupancy = valid_occupancy.median()
    
    print(f"\nOcupação por unidade habitacional:")
    print(f"- Média: {mean_occupancy:.2f}")
    print(f"- Mediana: {median_occupancy:.2f}")

# Estatísticas por classe de densidade
if 'density_class' in buildings_with_density.columns:
    print("\nDistribuição por classe de densidade:")
    class_stats = buildings_with_density['density_class'].value_counts()
    for cls, count in class_stats.items():
        print(f"- {cls}: {count} edifícios ({count/buildings_with_pop*100:.2f}% dos habitados)")

# Estatísticas da grade, se disponível
if grid_density is not None and 'population_density' in grid_density.columns:
    cells_with_pop = grid_density[grid_density['population'] > 0]
    mean_cell_density = cells_with_pop['population_density'].mean()
    max_cell_density = cells_with_pop['population_density'].max()
    
    print(f"\nDensidade populacional por célula (hab/km²):")
    print(f"- Células habitadas: {len(cells_with_pop)} de {len(grid_density)} ({len(cells_with_pop)/len(grid_density)*100:.2f}%)")
    print(f"- Média: {mean_cell_density:.2f}")
    print(f"- Máxima: {max_cell_density:.2f}")

# Benefícios das métricas calculadas
print("\nBenefícios das métricas de densidade habitacional:")
print("1. Identificação de áreas de maior pressão populacional")
print("2. Suporte para dimensionamento de infraestrutura e serviços urbanos")
print("3. Base para análises de impacto na qualidade de vida urbana")
print("4. Parâmetro para modelagem de ocupação temporal e fluxos populacionais")

# Próximos passos
print("\nPróximos passos:")
print("1. Modelagem de padrões temporais de ocupação")
print("2. Análise de correlação entre densidade e outras variáveis urbanas")
print("3. Projeções de crescimento populacional por área")

print(f"\nOs resultados foram salvos e estão prontos para o próximo notebook:")
print(f"- Edifícios com métricas de densidade: {os.path.basename(buildings_paths[0])}")
print(f"- Grade com métricas de densidade: {os.path.basename(grid_paths[0]) if grid_paths else 'Não gerado'}")
print("="*80)

3.3_Modelagem_Padroes_Temporais.ipynb

In [ ]:
# Modelagem de Padrões Temporais de Ocupação
# Este notebook estima padrões temporais de ocupação de edifícios ao longo do dia

import os
import pickle
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import contextily as cx
import warnings
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.dates as mdates
from datetime import datetime, timedelta

# Ignorar avisos específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Definição dos diretórios
data_dir = '/content/drive/MyDrive/geoprocessamento_gnn/DATA'
results_dir = os.path.join(data_dir, 'intermediate_results')
buildings_density_path = os.path.join(results_dir, 'buildings_density.pkl')

# Verificando se arquivos necessários existem
if not os.path.exists(buildings_density_path):
    raise FileNotFoundError(f"Arquivo não encontrado: {buildings_density_path}. Execute o notebook 3.2_Calculo_Densidade_Habitacional.ipynb primeiro.")

# Carregando os dados de edifícios com métricas de densidade
with open(buildings_density_path, 'rb') as f:
    buildings = pickle.load(f)

print(f"Dados carregados: {len(buildings)} edifícios com métricas de densidade.")

In [ ]:
# Analisar os dados de edifícios para identificar padrões de uso/ocupação
def analyze_building_uses(buildings_gdf):
    """
    Analisa os dados de edifícios para identificar padrões de uso/ocupação.
    
    Args:
        buildings_gdf: GeoDataFrame com os edifícios
    
    Returns:
        Dicionário com informações sobre tipos de uso e categorias de edifícios
    """
    # Identificar colunas que podem conter informações de uso/tipo
    use_columns = [col for col in buildings_gdf.columns if any(keyword in col.lower() for keyword in 
                                                             ['building', 'type', 'class', 'uso', 'categor', 'func'])]
    
    print(f"Colunas potenciais de uso/tipo: {use_columns}")
    
    # Selecionar as colunas mais promissoras
    selected_columns = []
    
    # Priorizar colunas específicas
    priority_cols = ['building_class_enhanced', 'categoria_funcional', 'building', 'building_class', 'tipo', 'type']
    
    for col in priority_cols:
        if col in use_columns and col in buildings_gdf.columns and buildings_gdf[col].notna().any():
            selected_columns.append(col)
    
    # Se não encontrou nenhuma coluna prioritária, usar qualquer uma disponível
    if not selected_columns and use_columns:
        for col in use_columns:
            if col in buildings_gdf.columns and buildings_gdf[col].notna().any():
                selected_columns.append(col)
                break
    
    if not selected_columns:
        raise ValueError("Não foi possível identificar colunas de uso/tipo dos edifícios.")
    
    # Analisar valores em cada coluna selecionada
    building_uses = {}
    
    for col in selected_columns:
        # Contar valores na coluna
        value_counts = buildings_gdf[col].value_counts()
        
        # Identificar categorias residenciais
        residential_keywords = ['residencial', 'residential', 'house', 'apartamento', 'apartments', 
                              'habitacional', 'dwelling', 'casa', 'moradia', 'housing']
        
        residential_values = []
        for value in value_counts.index:
            if pd.notna(value) and any(keyword in str(value).lower() for keyword in residential_keywords):
                residential_values.append(value)
        
        # Identificar categorias comerciais
        commercial_keywords = ['comercial', 'commercial', 'retail', 'shop', 'store', 'shopping', 'mercado',
                            'escritorio', 'office', 'business', 'serviço', 'service']
        
        commercial_values = []
        for value in value_counts.index:
            if pd.notna(value) and any(keyword in str(value).lower() for keyword in commercial_keywords):
                commercial_values.append(value)
        
        # Identificar categorias industriais
        industrial_keywords = ['industrial', 'industry', 'factory', 'fábrica', 'manufacturing', 'warehouse', 'storage']
        
        industrial_values = []
        for value in value_counts.index:
            if pd.notna(value) and any(keyword in str(value).lower() for keyword in industrial_keywords):
                industrial_values.append(value)
        
        # Identificar categorias institucionais
        institutional_keywords = ['institucional', 'institutional', 'education', 'school', 'escola', 'hospital', 
                               'health', 'saúde', 'government', 'governo', 'public', 'público']
        
        institutional_values = []
        for value in value_counts.index:
            if pd.notna(value) and any(keyword in str(value).lower() for keyword in institutional_keywords):
                institutional_values.append(value)
        
        # Adicionar ao dicionário
        building_uses[col] = {
            'values': value_counts,
            'residential': residential_values,
            'commercial': commercial_values,
            'industrial': industrial_values,
            'institutional': institutional_values
        }
    
    # Selecionar a melhor coluna para modelagem temporal
    best_column = None
    max_categories = 0
    
    for col, info in building_uses.items():
        # Contar categorias identificadas
        num_categories = (len(info['residential']) + len(info['commercial']) + 
                        len(info['industrial']) + len(info['institutional']))
        
        if num_categories > max_categories:
            max_categories = num_categories
            best_column = col
    
    # Exibir informações sobre a melhor coluna
    if best_column:
        print(f"\nColuna selecionada para modelagem temporal: {best_column}")
        print(f"Total de categorias identificadas: {max_categories}")
        
        # Mostrar distribuição de categorias
        for category, values in building_uses[best_column].items():
            if category != 'values':
                print(f"\nCategorias {category.upper()} identificadas ({len(values)}):")
                for value in values[:10]:  # Mostrar até 10 valores
                    count = building_uses[best_column]['values'][value]
                    print(f"- {value}: {count} edifícios")
                
                if len(values) > 10:
                    print(f"... (mais {len(values) - 10} categorias)")
        
        # Preparar informações para retorno
        return {
            'best_column': best_column,
            'categories': building_uses[best_column],
            'all_columns': selected_columns
        }
    else:
        return None

# Analisar usos dos edifícios
building_use_info = analyze_building_uses(buildings)

In [ ]:
# Definir perfis de ocupação temporal para diferentes categorias de edifícios
def define_temporal_profiles():
    """
    Define perfis de ocupação temporal para diferentes categorias de edifícios.
    
    Returns:
        Dicionário com perfis de ocupação horária por categoria
    """
    # Criar perfis de ocupação por hora do dia (0-23h)
    temporal_profiles = {
        # Perfil residencial (máximo à noite, mínimo durante horário comercial)
        'residential': {
            'hours': list(range(24)),  # 0 a 23 horas
            'occupancy_rate': [
                0.90, 0.95, 0.98, 0.99, 0.99, 0.95,  # 0h a 5h (noite/madrugada)
                0.85, 0.70, 0.50, 0.40, 0.35, 0.35,  # 6h a 11h (manhã)
                0.40, 0.45, 0.50, 0.55, 0.60, 0.65,  # 12h a 17h (tarde)
                0.75, 0.80, 0.85, 0.88, 0.89, 0.90   # 18h a 23h (noite)
            ],
            'description': 'Perfil típico de áreas residenciais, com ocupação máxima à noite e mínima durante o horário comercial.'
        },
        
        # Perfil comercial (máximo durante horário comercial, mínimo à noite)
        'commercial': {
            'hours': list(range(24)),
            'occupancy_rate': [
                0.05, 0.03, 0.02, 0.02, 0.03, 0.05,  # 0h a 5h (noite/madrugada)
                0.10, 0.30, 0.60, 0.85, 0.90, 0.95,  # 6h a 11h (manhã)
                0.98, 0.95, 0.90, 0.85, 0.80, 0.60,  # 12h a 17h (tarde)
                0.40, 0.30, 0.20, 0.15, 0.10, 0.05   # 18h a 23h (noite)
            ],
            'description': 'Perfil típico de áreas comerciais, com ocupação máxima durante o horário comercial e mínima à noite.'
        },
        
        # Perfil industrial (máximo durante turnos de trabalho)
        'industrial': {
            'hours': list(range(24)),
            'occupancy_rate': [
                0.30, 0.30, 0.30, 0.30, 0.30, 0.40,  # 0h a 5h (madrugada, 1º turno)
                0.60, 0.90, 0.95, 0.95, 0.95, 0.95,  # 6h a 11h (manhã, 2º turno)
                0.95, 0.95, 0.95, 0.95, 0.95, 0.90,  # 12h a 17h (tarde, 2º/3º turno)
                0.60, 0.40, 0.30, 0.30, 0.30, 0.30   # 18h a 23h (noite, 3º/1º turno)
            ],
            'description': 'Perfil típico de áreas industriais, com operação em múltiplos turnos e ocupação mais constante ao longo do dia.'
        },
        
        # Perfil institucional (máximo durante horário comercial estendido)
        'institutional': {
            'hours': list(range(24)),
            'occupancy_rate': [
                0.10, 0.05, 0.05, 0.05, 0.05, 0.10,  # 0h a 5h (noite/madrugada)
                0.20, 0.60, 0.90, 0.95, 0.98, 0.98,  # 6h a 11h (manhã)
                0.95, 0.95, 0.95, 0.90, 0.80, 0.50,  # 12h a 17h (tarde)
                0.30, 0.25, 0.20, 0.15, 0.12, 0.10   # 18h a 23h (noite)
            ],
            'description': 'Perfil típico de instituições como escolas, hospitais e órgãos públicos, com ocupação máxima durante o horário de funcionamento.'
        },
        
        # Perfil misto (combinação de residencial e comercial)
        'mixed': {
            'hours': list(range(24)),
            'occupancy_rate': [
                0.50, 0.50, 0.50, 0.50, 0.50, 0.55,  # 0h a 5h
                0.60, 0.70, 0.75, 0.80, 0.85, 0.90,  # 6h a 11h
                0.90, 0.85, 0.80, 0.80, 0.75, 0.70,  # 12h a 17h
                0.65, 0.60, 0.55, 0.50, 0.50, 0.50   # 18h a 23h
            ],
            'description': 'Perfil para áreas de uso misto, combinando características residenciais e comerciais.'
        },
        
        # Perfil padrão/desconhecido (ocupação moderada durante o dia)
        'default': {
            'hours': list(range(24)),
            'occupancy_rate': [
                0.25, 0.20, 0.20, 0.20, 0.25, 0.30,  # 0h a 5h
                0.40, 0.60, 0.70, 0.80, 0.85, 0.85,  # 6h a 11h
                0.85, 0.85, 0.80, 0.75, 0.70, 0.60,  # 12h a 17h
                0.50, 0.40, 0.35, 0.30, 0.25, 0.25   # 18h a 23h
            ],
            'description': 'Perfil padrão para edifícios sem categoria específica.'
        }
    }
    
    # Adicionar perfis específicos para subcategorias
    
    # Escolas (ocupação concentrada no período diurno)
    temporal_profiles['school'] = {
        'hours': list(range(24)),
        'occupancy_rate': [
            0.00, 0.00, 0.00, 0.00, 0.00, 0.05,  # 0h a 5h
            0.20, 0.80, 0.95, 0.98, 0.98, 0.95,  # 6h a 11h
            0.90, 0.95, 0.95, 0.80, 0.40, 0.10,  # 12h a 17h
            0.05, 0.02, 0.00, 0.00, 0.00, 0.00   # 18h a 23h
        ],
        'description': 'Perfil específico para escolas, com ocupação concentrada no período letivo.'
    }
    
    # Hospitais (ocupação constante com variações moderadas)
    temporal_profiles['hospital'] = {
        'hours': list(range(24)),
        'occupancy_rate': [
            0.60, 0.55, 0.50, 0.50, 0.55, 0.60,  # 0h a 5h
            0.70, 0.85, 0.95, 0.98, 0.98, 0.95,  # 6h a 11h
            0.90, 0.95, 0.95, 0.90, 0.85, 0.80,  # 12h a 17h
            0.75, 0.70, 0.65, 0.60, 0.60, 0.60   # 18h a 23h
        ],
        'description': 'Perfil específico para hospitais, com ocupação constante e picos durante horários de visita e atendimento ambulatorial.'
    }
    
    # Comércio varejista (concentrado em horário comercial + noite)
    temporal_profiles['retail'] = {
        'hours': list(range(24)),
        'occupancy_rate': [
            0.00, 0.00, 0.00, 0.00, 0.00, 0.00,  # 0h a 5h
            0.05, 0.30, 0.65, 0.85, 0.90, 0.95,  # 6h a 11h
            0.98, 0.95, 0.90, 0.95, 0.98, 0.90,  # 12h a 17h
            0.80, 0.50, 0.20, 0.05, 0.00, 0.00   # 18h a 23h
        ],
        'description': 'Perfil específico para comércio varejista, com ocupação concentrada no horário comercial e início da noite.'
    }
    
    # Escritórios (horário comercial padrão)
    temporal_profiles['office'] = {
        'hours': list(range(24)),
        'occupancy_rate': [
            0.02, 0.01, 0.01, 0.01, 0.02, 0.05,  # 0h a 5h
            0.20, 0.60, 0.90, 0.95, 0.95, 0.90,  # 6h a 11h
            0.95, 0.98, 0.95, 0.90, 0.70, 0.40,  # 12h a 17h
            0.20, 0.15, 0.10, 0.05, 0.02, 0.02   # 18h a 23h
        ],
        'description': 'Perfil específico para escritórios, com ocupação concentrada no horário comercial padrão.'
    }
    
    # Restaurantes e bares (picos no almoço e jantar)
    temporal_profiles['restaurant'] = {
        'hours': list(range(24)),
        'occupancy_rate': [
            0.40, 0.20, 0.05, 0.01, 0.00, 0.05,  # 0h a 5h
            0.10, 0.20, 0.30, 0.50, 0.80, 0.95,  # 6h a 11h
            0.90, 0.60, 0.40, 0.40, 0.60, 0.85,  # 12h a 17h
            0.95, 0.98, 0.95, 0.85, 0.70, 0.55   # 18h a 23h
        ],
        'description': 'Perfil específico para restaurantes e bares, com picos nos horários de refeições e à noite.'
    }
    
    return temporal_profiles

# Definir perfis de ocupação temporal
temporal_profiles = define_temporal_profiles()

# Visualizar perfis de ocupação
def visualize_temporal_profiles(profiles):
    """
    Visualiza os perfis de ocupação temporal definidos.
    
    Args:
        profiles: Dicionário com perfis de ocupação temporal
    """
    # Criar figura para visualização
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Definir cores para cada perfil
    colors = {
        'residential': '#1b9e77',
        'commercial': '#d95f02',
        'industrial': '#7570b3',
        'institutional': '#e7298a',
        'mixed': '#66a61e',
        'default': '#999999',
        'school': '#e6ab02',
        'hospital': '#a6761d',
        'retail': '#e41a1c',
        'office': '#377eb8',
        'restaurant': '#984ea3'
    }
    
    # Plotar cada perfil
    for name, profile in profiles.items():
        if 'hours' in profile and 'occupancy_rate' in profile:
            ax.plot(profile['hours'], profile['occupancy_rate'], 
                   label=name, linewidth=2, marker='o', markersize=4,
                   color=colors.get(name, None))
    
    # Configurar gráfico
    ax.set_xticks(range(0, 24, 2))
    ax.set_xlim(-0.5, 23.5)
    ax.set_ylim(0, 1.05)
    ax.set_xlabel('Hora do Dia', fontsize=12)
    ax.set_ylabel('Taxa de Ocupação', fontsize=12)
    ax.set_title('Perfis de Ocupação Temporal por Categoria de Edifício', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Adicionar linhas de grade para as horas
    for h in range(0, 24, 6):
        ax.axvline(x=h, color='gray', linestyle='--', alpha=0.5)
    
    # Adicionar períodos do dia
    ax.annotate('Madrugada', xy=(3, 1.02), ha='center', va='bottom', fontsize=10)
    ax.annotate('Manhã', xy=(9, 1.02), ha='center', va='bottom', fontsize=10)
    ax.annotate('Tarde', xy=(15, 1.02), ha='center', va='bottom', fontsize=10)
    ax.annotate('Noite', xy=(21, 1.02), ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Visualizar perfis temporais
visualize_temporal_profiles(temporal_profiles)

In [ ]:
# Associar categorias de edifícios aos perfis de ocupação temporal
def map_building_types_to_profiles(buildings_gdf, use_info, profiles):
    """
    Associa categorias de edifícios aos perfis de ocupação temporal.
    
    Args:
        buildings_gdf: GeoDataFrame com os edifícios
        use_info: Informações sobre categorias de uso dos edifícios
        profiles: Dicionário com perfis de ocupação temporal
    
    Returns:
        GeoDataFrame com perfil temporal associado a cada edifício
    """
    # Copiar o GeoDataFrame para não modificar o original
    buildings_with_profiles = buildings_gdf.copy()
    
    # Adicionar coluna para o perfil temporal
    buildings_with_profiles['temporal_profile'] = 'default'
    
    if not use_info or 'best_column' not in use_info:
        print("Informações de uso não disponíveis. Usando perfil padrão para todos os edifícios.")
        return buildings_with_profiles
    
    # Obter a melhor coluna para categorização
    best_column = use_info['best_column']
    categories = use_info['categories']
    
    # Mapeamento específico de subcategorias para perfis mais detalhados
    specific_mappings = {
        'school': ['school', 'education', 'educational', 'university', 'college', 'academia'],
        'hospital': ['hospital', 'healthcare', 'clinic', 'medical', 'saude', 'health'],
        'retail': ['retail', 'shop', 'store', 'shopping', 'mercado', 'loja', 'varejo'],
        'office': ['office', 'escritorio', 'business', 'corporate', 'empresa'],
        'restaurant': ['restaurant', 'bar', 'cafe', 'food', 'restaurante']
    }
    
    # Contador para estatísticas
    profile_counts = {name: 0 for name in profiles.keys()}
    
    # Mapear cada edifício para um perfil temporal
    for idx, row in tqdm(buildings_with_profiles.iterrows(), total=len(buildings_with_profiles), desc="Associando perfis temporais"):
        building_type = row.get(best_column)
        
        if pd.isna(building_type):
            # Se não tiver categoria, usar perfil padrão
            buildings_with_profiles.at[idx, 'temporal_profile'] = 'default'
            profile_counts['default'] += 1
            continue
        
        building_type_str = str(building_type).lower()
        
        # Primeiro verificar mapeamentos específicos
        mapped = False
        for profile_name, keywords in specific_mappings.items():
            if any(keyword in building_type_str for keyword in keywords):
                buildings_with_profiles.at[idx, 'temporal_profile'] = profile_name
                profile_counts[profile_name] += 1
                mapped = True
                break
        
        if mapped:
            continue
        
        # Verificar categorias gerais
        for category, values in categories.items():
            if category != 'values' and building_type in values:
                buildings_with_profiles.at[idx, 'temporal_profile'] = category
                profile_counts[category] += 1
                mapped = True
                break
        
       
# Se ainda não foi mapeado, tentar correspondência por palavra-chave
        if not mapped:
            if any(keyword in building_type_str for keyword in ['resid', 'habit', 'house', 'apart', 'dwelling']):
                buildings_with_profiles.at[idx, 'temporal_profile'] = 'residential'
                profile_counts['residential'] += 1
            elif any(keyword in building_type_str for keyword in ['comer', 'shop', 'loja', 'retail', 'mercado']):
                buildings_with_profiles.at[idx, 'temporal_profile'] = 'commercial'
                profile_counts['commercial'] += 1
            elif any(keyword in building_type_str for keyword in ['indus', 'factor', 'fabri', 'manufact', 'warehouse']):
                buildings_with_profiles.at[idx, 'temporal_profile'] = 'industrial'
                profile_counts['industrial'] += 1
            elif any(keyword in building_type_str for keyword in ['instit', 'public', 'gov', 'escola', 'hospital']):
                buildings_with_profiles.at[idx, 'temporal_profile'] = 'institutional'
                profile_counts['institutional'] += 1
            elif any(keyword in building_type_str for keyword in ['mix', 'misto']):
                buildings_with_profiles.at[idx, 'temporal_profile'] = 'mixed'
                profile_counts['mixed'] += 1
            else:
                # Se não conseguiu mapear para nenhum perfil específico, usar padrão
                buildings_with_profiles.at[idx, 'temporal_profile'] = 'default'
                profile_counts['default'] += 1
    
    # Exibir estatísticas de associação
    print("\nDistribuição de edifícios por perfil temporal:")
    total_buildings = len(buildings_with_profiles)
    for profile, count in profile_counts.items():
        if count > 0:
            print(f"- {profile}: {count} edifícios ({count/total_buildings*100:.2f}%)")
    
    return buildings_with_profiles

# Associar edifícios a perfis temporais
buildings_with_profiles = map_building_types_to_profiles(buildings, building_use_info, temporal_profiles)               buildings

In [ ]:
# Calcular ocupação populacional horária
def calculate_hourly_occupancy(buildings_gdf, profiles):
    """
    Calcula a ocupação populacional horária para cada edifício.
    
    Args:
        buildings_gdf: GeoDataFrame com edifícios e perfis temporais
        profiles: Dicionário com perfis de ocupação temporal
    
    Returns:
        GeoDataFrame com ocupação populacional horária
    """
    # Copiar o GeoDataFrame para não modificar o original
    buildings_hourly = buildings_gdf.copy()
    
    # Verificar se coluna de população estimada existe
    if 'estimated_population' not in buildings_hourly.columns:
        raise ValueError("Coluna 'estimated_population' não encontrada. Não é possível calcular ocupação horária.")
    
    # Verificar se coluna de perfil temporal existe
    if 'temporal_profile' not in buildings_hourly.columns:
        raise ValueError("Coluna 'temporal_profile' não encontrada. Execute a função map_building_types_to_profiles primeiro.")
    
    # Criar colunas para ocupação horária (0 a 23h)
    for hour in range(24):
        column_name = f'population_h{hour:02d}'
        buildings_hourly[column_name] = 0.0
    
    # Calcular ocupação horária para cada edifício
    for idx, row in tqdm(buildings_hourly.iterrows(), total=len(buildings_hourly), desc="Calculando ocupação horária"):
        # Obter perfil temporal do edifício
        profile_name = row['temporal_profile']
        
        # Usar perfil padrão se o perfil especificado não existir
        if profile_name not in profiles:
            print(f"AVISO: Perfil '{profile_name}' não encontrado. Usando perfil padrão.")
            profile_name = 'default'
        
        # Obter população total do edifício
        total_population = row['estimated_population']
        
        # Ignorar edifícios sem população
        if pd.isna(total_population) or total_population <= 0:
            continue
        
        # Obter taxas de ocupação por hora
        occupancy_rates = profiles[profile_name]['occupancy_rate']
        
        # Calcular população para cada hora
        for hour in range(24):
            column_name = f'population_h{hour:02d}'
            buildings_hourly.at[idx, column_name] = total_population * occupancy_rates[hour]
    
    # Calcular estatísticas de ocupação horária
    hourly_totals = {}
    hourly_columns = [f'population_h{hour:02d}' for hour in range(24)]
    
    for column in hourly_columns:
        hourly_totals[column] = buildings_hourly[column].sum()
    
    # Exibir estatísticas
    print("\nEstatísticas de ocupação horária:")
    print(f"- População total estimada: {buildings_hourly['estimated_population'].sum():,.0f}")
    
    max_hour = max(hourly_totals, key=hourly_totals.get)
    min_hour = min(hourly_totals, key=hourly_totals.get)
    
    max_hour_num = int(max_hour.replace('population_h', ''))
    min_hour_num = int(min_hour.replace('population_h', ''))
    
    print(f"- Hora de pico: {max_hour_num:02d}h - {hourly_totals[max_hour]:,.0f} pessoas ({hourly_totals[max_hour]/buildings_hourly['estimated_population'].sum()*100:.2f}% do total)")
    print(f"- Hora de vale: {min_hour_num:02d}h - {hourly_totals[min_hour]:,.0f} pessoas ({hourly_totals[min_hour]/buildings_hourly['estimated_population'].sum()*100:.2f}% do total)")
    
    return buildings_hourly, hourly_totals

# Calcular ocupação horária
buildings_hourly, hourly_totals = calculate_hourly_occupancy(buildings_with_profiles, temporal_profiles)

In [ ]:
# Visualizar os padrões temporais de ocupação
def visualize_temporal_patterns(buildings_hourly, hourly_totals):
    """
    Visualiza os padrões temporais de ocupação populacional.
    
    Args:
        buildings_hourly: GeoDataFrame com ocupação horária
        hourly_totals: Dicionário com totais de ocupação por hora
    """
    # 1. Gráfico de linha com ocupação total por hora do dia
    plt.figure(figsize=(12, 6))
    
    # Extrair horas e totais
    hours = range(24)
    population_values = [hourly_totals[f'population_h{hour:02d}'] for hour in hours]
    
    # Criar gráfico
    plt.plot(hours, population_values, marker='o', linestyle='-', linewidth=2, markersize=8)
    
    # Adicionar grade e rótulos
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xlabel('Hora do Dia', fontsize=12)
    plt.ylabel('População Estimada', fontsize=12)
    plt.title('Padrão de Ocupação Total ao Longo do Dia', fontsize=14)
    
    # Formatar eixo x com horas
    plt.xticks(range(0, 24, 2))
    plt.xlim(-0.5, 23.5)
    
    # Adicionar rótulos para períodos do dia
    plt.annotate('Madrugada', xy=(3, max(population_values) * 1.02), ha='center', va='bottom', fontsize=10)
    plt.annotate('Manhã', xy=(9, max(population_values) * 1.02), ha='center', va='bottom', fontsize=10)
    plt.annotate('Tarde', xy=(15, max(population_values) * 1.02), ha='center', va='bottom', fontsize=10)
    plt.annotate('Noite', xy=(21, max(population_values) * 1.02), ha='center', va='bottom', fontsize=10)
    
    # Adicionar valor máximo e mínimo
    max_idx = population_values.index(max(population_values))
    min_idx = population_values.index(min(population_values))
    
    plt.annotate(f'Máximo: {max(population_values):,.0f}', xy=(max_idx, max(population_values)),
                xytext=(max_idx + 1, max(population_values) * 1.05),
                arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
                fontsize=10)
    
    plt.annotate(f'Mínimo: {min(population_values):,.0f}', xy=(min_idx, min(population_values)),
                xytext=(min_idx - 1, min(population_values) * 0.9),
                arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
                fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # 2. Gráfico de ocupação por perfil temporal
    # Agrupar por perfil e calcular ocupação por hora
    profiles_hourly = {}
    
    for profile in buildings_hourly['temporal_profile'].unique():
        if pd.isna(profile):
            continue
        
        profile_buildings = buildings_hourly[buildings_hourly['temporal_profile'] == profile]
        
        profile_hourly = []
        for hour in range(24):
            column = f'population_h{hour:02d}'
            profile_hourly.append(profile_buildings[column].sum())
        
        profiles_hourly[profile] = profile_hourly
    
    # Criar gráfico por perfil
    plt.figure(figsize=(14, 8))
    
    # Definir cores para cada perfil
    colors = {
        'residential': '#1b9e77',
        'commercial': '#d95f02',
        'industrial': '#7570b3',
        'institutional': '#e7298a',
        'mixed': '#66a61e',
        'default': '#999999',
        'school': '#e6ab02',
        'hospital': '#a6761d',
        'retail': '#e41a1c',
        'office': '#377eb8',
        'restaurant': '#984ea3'
    }
    
    # Plotar cada perfil
    for profile, hourly_data in profiles_hourly.items():
        if sum(hourly_data) > 0:  # Só plotar perfis com dados
            plt.plot(range(24), hourly_data, label=profile, linewidth=2, marker='o',
                   color=colors.get(profile, None))
    
    # Configurar gráfico
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xlabel('Hora do Dia', fontsize=12)
    plt.ylabel('População Estimada', fontsize=12)
    plt.title('Padrões de Ocupação por Perfil Temporal', fontsize=14)
    plt.xticks(range(0, 24, 2))
    plt.xlim(-0.5, 23.5)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.tight_layout()
    plt.show()
    
    # 3. Heatmap da distribuição horária por perfil
    # Preparar dados para o heatmap
    heatmap_data = []
    profiles_list = []
    
    for profile, hourly_data in profiles_hourly.items():
        if sum(hourly_data) > 0:  # Só incluir perfis com dados
            # Normalizar pela ocupação máxima do perfil para o heatmap
            max_value = max(hourly_data)
            if max_value > 0:
                normalized_data = [value / max_value for value in hourly_data]
                heatmap_data.append(normalized_data)
                profiles_list.append(profile)
    
    # Criar heatmap
    plt.figure(figsize=(14, 8))
    
    # Plotar heatmap
    ax = sns.heatmap(heatmap_data, cmap='viridis', 
                    xticklabels=range(0, 24, 2),
                    yticklabels=profiles_list,
                    annot=False, cbar_kws={'label': 'Ocupação Relativa'})
    
    # Configurar eixos
    ax.set_xlabel('Hora do Dia', fontsize=12)
    ax.set_ylabel('Perfil Temporal', fontsize=12)
    ax.set_title('Distribuição Horária por Perfil (Normalizada)', fontsize=14)
    
    plt.tight_layout()
    plt.show()

# Visualizar padrões temporais
visualize_temporal_patterns(buildings_hourly, hourly_totals)

In [ ]:
# Visualizar mapas de ocupação para diferentes horas do dia
def visualize_hourly_maps(buildings_hourly, selected_hours=None):
    """
    Cria mapas de ocupação para diferentes horas do dia.
    
    Args:
        buildings_hourly: GeoDataFrame com ocupação horária
        selected_hours: Lista de horas para visualizar (None para usar horas padrão)
    """
    # Se não informar horas específicas, usar padrão (4h, 8h, 12h, 16h, 20h)
    if selected_hours is None:
        selected_hours = [4, 8, 12, 16, 20]
    
    # Verificar se são horas válidas
    selected_hours = [h for h in selected_hours if 0 <= h < 24]
    
    if not selected_hours:
        print("Nenhuma hora válida selecionada.")
        return
    
    # Criar um grid de subplots (até 6 mapas por figura)
    max_maps_per_fig = 6
    num_figs = (len(selected_hours) + max_maps_per_fig - 1) // max_maps_per_fig
    
    for fig_num in range(num_figs):
        # Selecionar horas para esta figura
        start_idx = fig_num * max_maps_per_fig
        end_idx = min(start_idx + max_maps_per_fig, len(selected_hours))
        hours_subset = selected_hours[start_idx:end_idx]
        
        # Calcular layout de subplots
        if len(hours_subset) <= 3:
            rows, cols = 1, len(hours_subset)
        else:
            rows = (len(hours_subset) + 1) // 2
            cols = 2
        
        # Criar figura
        fig, axs = plt.subplots(rows, cols, figsize=(cols * 8, rows * 8))
        
        # Converter para array se necessário (para indexação consistente)
        if rows == 1 and cols == 1:
            axs = np.array([axs])
        elif rows == 1 or cols == 1:
            axs = axs.ravel()
        
        # Plotar mapa para cada hora
        for i, hour in enumerate(hours_subset):
            column_name = f'population_h{hour:02d}'
            
            # Filtrar edifícios com ocupação nesta hora
            buildings_with_pop = buildings_hourly[buildings_hourly[column_name] > 0]
            
            # Determinar tamanho dos pontos baseado na população
            if len(buildings_with_pop) > 0:
                max_pop = buildings_with_pop[column_name].max()
                min_pop = buildings_with_pop[column_name].min()
                
                if max_pop > min_pop:
                    buildings_with_pop['point_size'] = buildings_with_pop[column_name].map(
                        lambda p: 5 + (p - min_pop) / (max_pop - min_pop) * 95
                    )
                else:
                    buildings_with_pop['point_size'] = 10
            
            # Determinar índice do subplot
            ax = axs[i // cols, i % cols] if rows > 1 and cols > 1 else axs[i]
            
            # Plotar mapa
            if len(buildings_with_pop) > 0:
                buildings_with_pop.plot(column=column_name, cmap='plasma',
                                      markersize='point_size', alpha=0.7,
                                      ax=ax, legend=True,
                                      legend_kwds={'label': 'População',
                                                  'orientation': 'horizontal',
                                                  'shrink': 0.8,
                                                  'pad': 0.01})
            else:
                # Se não houver dados, plotar um mapa vazio
                buildings_hourly.plot(ax=ax, color='lightgray', alpha=0.3)
            
            # Adicionar mapa base
            try:
                cx.add_basemap(ax, crs=buildings_hourly.crs)
            except Exception as e:
                print(f"Não foi possível adicionar o mapa base para hora {hour}: {e}")
            
            # Adicionar título
            ax.set_title(f'Ocupação às {hour:02d}:00h', fontsize=14)
            
            # Adicionar texto com estatísticas
            if column_name in hourly_totals:
                total_pop = hourly_totals[column_name]
                max_pop = hourly_totals[max(hourly_totals, key=hourly_totals.get)]
                
                ax.annotate(f'População: {total_pop:,.0f} ({total_pop/max_pop*100:.1f}% do pico)',
                          xy=(0.02, 0.02), xycoords='axes fraction',
                          bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
                          fontsize=10)
        
        # Ajustar layout
        plt.tight_layout()
        plt.show()

# Visualizar mapas para diferentes horas do dia
# Selecionar horas representativas (madrugada, manhã, almoço, tarde, noite)
selected_hours = [3, 8, 12, 17, 21]
visualize_hourly_maps(buildings_hourly, selected_hours)

In [ ]:
# Calcular métricas do pulso urbano
def calculate_urban_pulse_metrics(buildings_hourly, hourly_totals):
    """
    Calcula métricas que caracterizam o pulso urbano.
    
    Args:
        buildings_hourly: GeoDataFrame com ocupação horária
        hourly_totals: Dicionário com totais de ocupação por hora
    
    Returns:
        DataFrame com métricas de pulso urbano
    """
    # Preparar dicionário para armazenar métricas
    pulse_metrics = {}
    
    # 1. Amplitude do pulso urbano (diferença entre ocupação máxima e mínima)
    max_hour = max(hourly_totals, key=hourly_totals.get)
    min_hour = min(hourly_totals, key=hourly_totals.get)
    
    amplitude = hourly_totals[max_hour] - hourly_totals[min_hour]
    amplitude_pct = amplitude / hourly_totals[max_hour] * 100
    
    pulse_metrics['amplitude_abs'] = amplitude
    pulse_metrics['amplitude_pct'] = amplitude_pct
    
    # 2. Horas de pico e vale
    max_hour_num = int(max_hour.replace('population_h', ''))
    min_hour_num = int(min_hour.replace('population_h', ''))
    
    pulse_metrics['peak_hour'] = max_hour_num
    pulse_metrics['valley_hour'] = min_hour_num
    
    # 3. Índice de atividade diurna (razão entre média diurna e média total)
    daytime_hours = range(8, 20)  # 8h às 19h
    night_hours = list(range(0, 8)) + list(range(20, 24))  # 20h às 7h
    
    daytime_totals = [hourly_totals[f'population_h{hour:02d}'] for hour in daytime_hours]
    night_totals = [hourly_totals[f'population_h{hour:02d}'] for hour in night_hours]
    
    daytime_avg = sum(daytime_totals) / len(daytime_totals)
    night_avg = sum(night_totals) / len(night_totals)
    total_avg = sum(hourly_totals.values()) / len(hourly_totals)
    
    pulse_metrics['daytime_activity_index'] = daytime_avg / total_avg
    pulse_metrics['night_activity_index'] = night_avg / total_avg
    pulse_metrics['day_night_ratio'] = daytime_avg / night_avg if night_avg > 0 else float('inf')
    
    # 4. Períodos de atividade (horas consecutivas acima de um limiar)
    # Definir limiar como 80% da ocupação máxima
    threshold = hourly_totals[max_hour] * 0.8
    
    active_hours = [hour for hour, total in hourly_totals.items() if total >= threshold]
    active_hours_nums = [int(hour.replace('population_h', '')) for hour in active_hours]
    active_hours_nums.sort()
    
    # Identificar períodos consecutivos
    active_periods = []
    current_period = []
    
    for i, hour in enumerate(active_hours_nums):
        if i == 0 or hour != active_hours_nums[i-1] + 1:
            if current_period:
                active_periods.append(current_period)
            current_period = [hour]
        else:
            current_period.append(hour)
    
    if current_period:
        active_periods.append(current_period)
    
    pulse_metrics['num_active_periods'] = len(active_periods)
    pulse_metrics['longest_active_period'] = max([len(period) for period in active_periods]) if active_periods else 0
    
    # Formatar períodos para exibição
    active_periods_str = []
    for period in active_periods:
        if len(period) == 1:
            active_periods_str.append(f"{period[0]:02d}h")
        else:
            active_periods_str.append(f"{period[0]:02d}h-{period[-1]:02d}h")
    
    pulse_metrics['active_periods'] = ", ".join(active_periods_str)
    
    # 5. Índice de comutação (relacionado à movimentação entre áreas residenciais e não residenciais)
    residential_buildings = buildings_hourly[buildings_hourly['temporal_profile'] == 'residential']
    nonresidential_buildings = buildings_hourly[buildings_hourly['temporal_profile'] != 'residential']
    
    # Calcular ocupação por hora para cada tipo
    residential_hourly = {}
    nonresidential_hourly = {}
    
    for hour in range(24):
        column_name = f'population_h{hour:02d}'
        
        if not residential_buildings.empty:
            residential_hourly[hour] = residential_buildings[column_name].sum()
        else:
            residential_hourly[hour] = 0
        
        if not nonresidential_buildings.empty:
            nonresidential_hourly[hour] = nonresidential_buildings[column_name].sum()
        else:
            nonresidential_hourly[hour] = 0
    
    # Calcular índice de comutação (correlação negativa entre ocupação residencial e não residencial)
    if len(residential_hourly) > 0 and len(nonresidential_hourly) > 0:
        res_values = list(residential_hourly.values())
        nonres_values = list(nonresidential_hourly.values())
        
        # Calcular correlação de Pearson
        if np.std(res_values) > 0 and np.std(nonres_values) > 0:
            correlation = np.corrcoef(res_values, nonres_values)[0, 1]
            pulse_metrics['commuting_index'] = -correlation  # Invertido para que valores positivos indiquem mais comutação
        else:
            pulse_metrics['commuting_index'] = 0
    else:
        pulse_metrics['commuting_index'] = 0
    
    # Exibir métricas calculadas
    print("Métricas do Pulso Urbano:")
    print(f"- Amplitude: {amplitude:,.0f} pessoas ({amplitude_pct:.2f}% do pico)")
    print(f"- Hora de pico: {max_hour_num:02d}h")
    print(f"- Hora de vale: {min_hour_num:02d}h")
    print(f"- Índice de atividade diurna: {pulse_metrics['daytime_activity_index']:.4f}")
    print(f"- Índice de atividade noturna: {pulse_metrics['night_activity_index']:.4f}")
    print(f"- Razão dia/noite: {pulse_metrics['day_night_ratio']:.4f}")
    print(f"- Períodos de alta atividade: {pulse_metrics['active_periods']}")
    print(f"- Índice de comutação: {pulse_metrics['commuting_index']:.4f}")
    
    # Criar DataFrame para facilitar armazenamento e retorno
    metrics_df = pd.DataFrame([pulse_metrics])
    
    return metrics_df, residential_hourly, nonresidential_hourly

# Calcular métricas de pulso urbano
pulse_metrics, residential_hourly, nonresidential_hourly = calculate_urban_pulse_metrics(buildings_hourly, hourly_totals)

In [ ]:
# Visualizar o pulso urbano e padrões de comutação
def visualize_urban_pulse(residential_hourly, nonresidential_hourly, pulse_metrics):
    """
    Visualiza o pulso urbano e padrões de comutação.
    
    Args:
        residential_hourly: Dicionário com ocupação residencial por hora
        nonresidential_hourly: Dicionário com ocupação não residencial por hora
        pulse_metrics: DataFrame com métricas de pulso urbano
    """
    # 1. Gráfico de comutação (ocupação residencial x não residencial)
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    # Eixo para ocupação residencial
    color1 = '#1b9e77'
    ax1.set_xlabel('Hora do Dia', fontsize=12)
    ax1.set_ylabel('Ocupação Residencial', fontsize=12, color=color1)
    ax1.plot(list(residential_hourly.keys()), list(residential_hourly.values()), 
            marker='o', linestyle='-', linewidth=2, color=color1, label='Residencial')
    ax1.tick_params(axis='y', labelcolor=color1)
    
    # Adicionar segundo eixo para ocupação não residencial
    ax2 = ax1.twinx()
    color2 = '#d95f02'
    ax2.set_ylabel('Ocupação Não Residencial', fontsize=12, color=color2)
    ax2.plot(list(nonresidential_hourly.keys()), list(nonresidential_hourly.values()), 
            marker='s', linestyle='-', linewidth=2, color=color2, label='Não Residencial')
    ax2.tick_params(axis='y', labelcolor=color2)
    
    # Ajustar configurações do gráfico
    plt.title('Padrões de Comutação: Ocupação Residencial x Não Residencial', fontsize=14)
    ax1.set_xticks(range(0, 24, 2))
    ax1.set_xlim(-0.5, 23.5)
    ax1.grid(True, linestyle='--', alpha=0.7)
    
    # Adicionar legenda combinada
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
    
    # Adicionar anotação com índice de comutação
    commuting_index = pulse_metrics['commuting_index'].iloc[0]
    plt.annotate(f"Índice de Comutação: {commuting_index:.4f}",
               xy=(0.02, 0.02), xycoords='figure fraction',
               bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
               fontsize=10)
    
    # Marcar períodos de alta atividade
    if 'active_periods' in pulse_metrics.columns:
        active_periods_str = pulse_metrics['active_periods'].iloc[0]
        periods = active_periods_str.split(', ')
        
        for period in periods:
            if '-' in period:
                start, end = period.replace('h', '').split('-')
                start, end = int(start), int(end)
                plt.axvspan(start, end + 1, alpha=0.2, color='gray')
            else:
                hour = int(period.replace('h', ''))
                plt.axvspan(hour, hour + 1, alpha=0.2, color='gray')
    
    plt.tight_layout()
    plt.show()
    
    # 2. Visualização do ciclo de 24 horas como um relógio
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': 'polar'})
    
    # Preparar dados
    hours = np.array(list(range(24)) + [0]) * 2 * np.pi / 24  # Converter para radianos e fechar o ciclo
    
    # Ocupação total
    total_values = [hourly_totals[f'population_h{h:02d}'] for h in range(24)]
    total_values.append(total_values[0])  # Fechar o ciclo
    
    # Normalizar para visualização
    max_total = max(total_values[:-1])
    normalized_total = [v / max_total for v in total_values]
    
    # Plotar ocupação total
    ax.plot(hours, normalized_total, 'o-', linewidth=3, markersize=8, label='Ocupação Total')
    
    # Ocupação residencial
    res_values = list(residential_hourly.values())
    res_values.append(res_values[0])  # Fechar o ciclo
    
    # Normalizar
    max_res = max(res_values[:-1])
    normalized_res = [v / max_res for v in res_values]
    
    # Plotar ocupação residencial
    ax.plot(hours, normalized_res, 's-', linewidth=2, markersize=6, label='Residencial')
    
    # Ocupação não residencial
    nonres_values = list(nonresidential_hourly.values())
    nonres_values.append(nonres_values[0])  # Fechar o ciclo
    
    # Normalizar
    max_nonres = max(nonres_values[:-1])
    normalized_nonres = [v / max_nonres for v in nonres_values]
    
    # Plotar ocupação não residencial
    ax.plot(hours, normalized_nonres, '^-', linewidth=2, markersize=6, label='Não Residencial')
    
    # Ajustar configurações do gráfico
    ax.set_theta_zero_location('top')  # 0h no topo
    ax.set_theta_direction(-1)  # Sentido horário
    
    # Configurar rótulos de horas
    ax.set_xticks(np.linspace(0, 2*np.pi, 24, endpoint=False))
    ax.set_xticklabels([f'{h:02d}h' for h in range(24)])
    
    # Remover rótulos do eixo radial
    ax.set_yticks([])
    
    # Adicionar título e legenda
    plt.title('Ciclo de Ocupação Urbana (24 horas)', fontsize=14, y=1.08)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)
    
    # Adicionar anotações para períodos do dia
    plt.annotate('Madrugada', xy=(0, 0), xytext=(0, 1.3), xycoords='data', 
               textcoords='axes fraction', ha='center', fontsize=10)
    plt.annotate('Manhã', xy=(np.pi/2, 0), xytext=(1.3, 0.5), xycoords='data', 
               textcoords='axes fraction', ha='center', va='center', fontsize=10)
    plt.annotate('Tarde', xy=(np.pi, 0), xytext=(0.5, -0.1), xycoords='data', 
               textcoords='axes fraction', ha='center', fontsize=10)
    plt.annotate('Noite', xy=(3*np.pi/2, 0), xytext=(-0.3, 0.5), xycoords='data', 
               textcoords='axes fraction', ha='center', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Visualizar pulso urbano
visualize_urban_pulse(residential_hourly, nonresidential_hourly, pulse_metrics)

In [ ]:
# Salvar resultados da modelagem temporal
def save_temporal_results(buildings_hourly, pulse_metrics):
    """
    Salva os resultados da modelagem temporal para uso em análises subsequentes.
    
    Args:
        buildings_hourly: GeoDataFrame com ocupação horária
        pulse_metrics: DataFrame com métricas de pulso urbano
    
    Returns:
        Caminhos para os arquivos salvos
    """
    # Criar pasta de resultados se não existir
    results_dir = os.path.join(data_dir, 'results')
    os.makedirs(results_dir, exist_ok=True)
    
    # Pasta para resultados intermediários
    intermediate_dir = os.path.join(data_dir, 'intermediate_results')
    os.makedirs(intermediate_dir, exist_ok=True)
    
    # Salvar GeoDataFrame com ocupação horária
    buildings_path = os.path.join(intermediate_dir, 'buildings_hourly.gpkg')
    buildings_hourly.to_file(buildings_path, driver='GPKG')
    
    # Salvar também como pickle para preservar tipos de dados
    buildings_pickle_path = os.path.join(intermediate_dir, 'buildings_hourly.pkl')
    with open(buildings_pickle_path, 'wb') as f:
        pickle.dump(buildings_hourly, f)
    
    # Salvar métricas de pulso urbano como CSV
    metrics_path = os.path.join(results_dir, 'urban_pulse_metrics.csv')
    pulse_metrics.to_csv(metrics_path, index=False)
    
    # Salvar dados horários agregados como CSV para facilitar análises externas
    hourly_data = {
        'hour': list(range(24)),
        'total_population': [hourly_totals[f'population_h{h:02d}'] for h in range(24)]
    }
    
    # Adicionar dados por tipo de perfil
    for profile in buildings_hourly['temporal_profile'].unique():
        if pd.isna(profile):
            continue
        
        profile_buildings = buildings_hourly[buildings_hourly['temporal_profile'] == profile]
        
        if len(profile_buildings) > 0:
            hourly_data[f'population_{profile}'] = [
                profile_buildings[f'population_h{h:02d}'].sum() for h in range(24)
            ]
    
    # Criar DataFrame e salvar como CSV
    hourly_df = pd.DataFrame(hourly_data)
    hourly_path = os.path.join(results_dir, 'hourly_population.csv')
    hourly_df.to_csv(hourly_path, index=False)
    
    print(f"Resultados da modelagem temporal salvos em:")
    print(f"- GeoPackage com ocupação horária: {buildings_path}")
    print(f"- Pickle com ocupação horária: {buildings_pickle_path}")
    print(f"- Métricas de pulso urbano: {metrics_path}")
    print(f"- Dados horários agregados: {hourly_path}")
    
    return buildings_path, metrics_path, hourly_path

# Salvar resultados
buildings_path, metrics_path, hourly_path = save_temporal_results(buildings_hourly, pulse_metrics)

In [ ]:
# Resumo e conclusão
print("="*80)
print("RESUMO DA MODELAGEM DE PADRÕES TEMPORAIS")
print("="*80)

# Estatísticas gerais
total_buildings = len(buildings_hourly)
buildings_with_profiles = buildings_hourly['temporal_profile'].notna().sum()
total_population = buildings_hourly['estimated_population'].sum()

print(f"Total de edifícios processados: {total_buildings}")
print(f"Edifícios com perfil temporal atribuído: {buildings_with_profiles} ({buildings_with_profiles/total_buildings*100:.2f}%)")
print(f"População total estimada: {total_population:,.0f}")

# Estatísticas do pulso urbano
max_hour = max(hourly_totals, key=hourly_totals.get)
min_hour = min(hourly_totals, key=hourly_totals.get)
max_hour_num = int(max_hour.replace('population_h', ''))
min_hour_num = int(min_hour.replace('population_h', ''))

print(f"\nPulso urbano:")
print(f"- Hora de pico: {max_hour_num:02d}h - {hourly_totals[max_hour]:,.0f} pessoas ({hourly_totals[max_hour]/total_population*100:.2f}% do total)")
print(f"- Hora de vale: {min_hour_num:02d}h - {hourly_totals[min_hour]:,.0f} pessoas ({hourly_totals[min_hour]/total_population*100:.2f}% do total)")
print(f"- Amplitude: {hourly_totals[max_hour] - hourly_totals[min_hour]:,.0f} pessoas ({(hourly_totals[max_hour] - hourly_totals[min_hour])/hourly_totals[max_hour]*100:.2f}%)")

# Estatísticas de comutação
if 'commuting_index' in pulse_metrics.columns:
    commuting_index = pulse_metrics['commuting_index'].iloc[0]
    print(f"- Índice de comutação: {commuting_index:.4f}")

# Períodos de atividade
if 'active_periods' in pulse_metrics.columns:
    active_periods = pulse_metrics['active_periods'].iloc[0]
    print(f"- Períodos de alta atividade: {active_periods}")

# Distribuição de perfis temporais
profile_counts = buildings_hourly['temporal_profile'].value_counts()

print("\nDistribuição de edifícios por perfil temporal:")
for profile, count in profile_counts.items():
    if pd.notna(profile):
        print(f"- {profile}: {count} edifícios ({count/total_buildings*100:.2f}%)")

# Benefícios da modelagem temporal
print("\nBenefícios da modelagem de padrões temporais:")
print("1. Compreensão da dinâmica urbana ao longo do dia")
print("2. Identificação de picos de ocupação para planejamento de serviços e infraestrutura")
print("3. Visualização de padrões de comutação entre áreas residenciais e não residenciais")
print("4. Base para modelagem de fluxos de mobilidade e impactos na rede viária")
print("5. Subsídio para planejamento de transportes e serviços públicos em diferentes períodos")

# Próximos passos
print("\nPróximos passos:")
print("1. Integração com dados de mobilidade para modelagem de fluxos")
print("2. Análise de acessibilidade a serviços essenciais em diferentes momentos do dia")
print("3. Simulação de cenários de ocupação para planejamento de emergências")
print("4. Modelagem de padrões semanais (dias úteis vs. fim de semana)")

print(f"\nOs resultados foram salvos e estão prontos para uso em outras análises:")
print(f"- GeoPackage com ocupação horária: {os.path.basename(buildings_path)}")
print(f"- Métricas de pulso urbano: {os.path.basename(metrics_path)}")
print(f"- Dados horários agregados: {os.path.basename(hourly_path)}")
print("="*80)